# CNN per embedding di Facce
Creazione di una CNN che presa un immagine in input la trasforma in un Embedding <br>
Obiettivo: faccie simili avranno vettori simili. <br>
Questo modello verrá usato nell'app di Riconoscimento Facciale per il Progetto di DeepLearning e Architetture di Reti Nurali UniBs 2025/2026

Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
import numpy as np
import zipfile
import shutil

# Download and Load Dataset functions

In [ ]:
def get_data_transforms(img_size=128, augment=True):
    """
    Crea le trasformazioni per train e test

    Args:
        img_size: dimensione output immagini
        augment: se True, applica data augmentation al training set

    Returns:
        train_transform, test_transform
    """
    # Trasformazioni base per test (solo resize e normalizzazione)
    test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Trasformazioni per training con augmentation
    if augment:
        train_transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        train_transform = test_transform

    return train_transform, test_transform


In [ ]:
def load_lfw_with_sklearn(data_dir='./content/data', min_faces_per_person=10):
    """
    Scarica LFW usando sklearn (metodo più affidabile)

    Args:
        data_dir: directory dove salvare il dataset
        min_faces_per_person: minimo immagini per persona

    Returns:
        images (numpy array), labels (numpy array), num_classes
    """
    print("Download LFW con sklearn...")
    lfw_people = fetch_lfw_people(
        data_home=data_dir,
        min_faces_per_person=min_faces_per_person,
        resize=1.0,  # mantiene dimensione originale
        color=True
    )

    images = lfw_people.images  # shape: (n_samples, height, width, 3)
    labels = lfw_people.target
    num_classes = len(lfw_people.target_names)

    print(f"Dataset scaricato:")
    print(f"  Totale samples: {len(images)}")
    print(f"  Dimensione immagini: {images[0].shape}")
    print(f"  Num classes: {num_classes}")

    return images, labels, num_classes


class LFWDataset(Dataset):
    """
    Custom Dataset per LFW da array numpy
    """
    def __init__(self, images, labels, transform=None):
        """
        Args:
            images: numpy array (n_samples, height, width, 3)
            labels: numpy array (n_samples,)
            transform: trasformazioni da applicare
        """
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Converti numpy array a PIL Image
        image = self.images[idx]
        # Assicurati che i valori siano in [0, 255]
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)
        image = Image.fromarray(image)

        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


def load_lfw_dataset(data_dir='./content/data', img_size=128, batch_size=64,
                     augment=True, min_faces_per_person=10, test_split=0.2):
    """
    Carica il dataset LFW usando sklearn

    Args:
        data_dir: directory dove salvare/caricare il dataset
        img_size: dimensione immagini
        batch_size: batch size per DataLoader
        augment: se True, applica data augmentation
        min_faces_per_person: minimo immagini per persona da includere
        test_split: percentuale per test set (es. 0.2 = 20%)

    Returns:
        train_loader, test_loader, num_classes
    """
    # Scarica dataset con sklearn
    images, labels, num_classes = load_lfw_with_sklearn(data_dir, min_faces_per_person)

    train_transform, test_transform = get_data_transforms(img_size, augment)

    # Split train/test stratificato
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels,
        test_size=test_split,
        stratify=labels,
        random_state=42
    )

    # Crea dataset con trasformazioni appropriate
    train_dataset = LFWDataset(train_images, train_labels, train_transform)
    test_dataset = LFWDataset(test_images, test_labels, test_transform)

    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    print(f"\nDataLoaders creati:")
    print(f"  Train samples: {len(train_dataset)}")
    print(f"  Test samples: {len(test_dataset)}")
    print(f"  Batch size: {batch_size}")

    return train_loader, test_loader, num_classes

# Creazione classi:
- FaceEmbeddingCNN come classe principale
- ResidualBlcok per avere un NiN


In [ ]:
class ResidualBlock(nn.Module):
    """
    Blocco residuale per CNN
    Conv → BN → ReLU → Conv → BN → (+skip) → ReLU
    """
    def __init__(self, in_channels, out_channels, kernel_size=3,
                 dropout_rate=0.5, use_batchnorm=True):
        super(ResidualBlock, self).__init__()

        padding = kernel_size // 2

        # Prima convoluzione
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                              kernel_size=kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity()
        self.relu1 = nn.ReLU()

        # Seconda convoluzione
        self.conv2 = nn.Conv2d(out_channels, out_channels,
                              kernel_size=kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity()

        # Projection shortcut se dimensioni diverse
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity()
            )
        else:
            self.shortcut = nn.Identity()

        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout2d(dropout_rate) if dropout_rate > 0 else nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Skip connection
        out = out + identity
        out = self.relu2(out)
        out = self.dropout(out)

        return out

In [ ]:
class FaceEmbeddingCNN(nn.Module):
    """
    Convolutional Neural Network per generare embeddings di volti.
    Durante training: classificazione delle identità (cross entropy)
    Durante inference: estrazione embedding dal penultimo layer

    Supporta sia CNN standard che CNN con residual connections (ResNet-style)
    """

    def __init__(self,
                 input_channels=3,  # RGB
                 num_filters=[32, 64, 128],  # filtri per ogni conv block
                 kernel_sizes=[3, 3, 3],  # kernel size per ogni conv block
                 fc_hidden_size=512,  # dimensione layer fully connected
                 embedding_size=128,  # dimensione embedding finale
                 num_classes=5749,  # numero identità
                 dropout_rate=0.5,  # dropout rate
                 use_batchnorm=True,  # usa batch normalization
                 use_global_avg_pool=False,  # usa global average pooling
                 use_residual=False):  # usa residual connections (skip connections)
        """
        Args:
            input_channels: numero canali input (3 per RGB)
            num_filters: lista con numero filtri per ogni conv block
            kernel_sizes: lista con kernel size per ogni conv block
            fc_hidden_size: dimensione layer fully connected dopo conv
            embedding_size: dimensione del vettore embedding
            num_classes: numero di identità (classi) per classificazione
            dropout_rate: probabilità dropout (0 = no dropout)
            use_batchnorm: se True, aggiunge BatchNorm dopo ogni conv
            use_global_avg_pool: se True, usa GAP invece di flatten
            use_residual: se True, usa residual blocks con skip connections
        """
        super(FaceEmbeddingCNN, self).__init__()

        self.embedding_size = embedding_size
        self.use_batchnorm = use_batchnorm
        self.use_global_avg_pool = use_global_avg_pool
        self.use_residual = use_residual

        # Costruzione convolutional blocks
        if use_residual:
            # Residual CNN con skip connections
            conv_layers = []
            in_channels = input_channels

            for num_filter, kernel_size in zip(num_filters, kernel_sizes):
                # Residual block
                conv_layers.append(ResidualBlock(
                    in_channels,
                    num_filter,
                    kernel_size=kernel_size,
                    dropout_rate=dropout_rate,
                    use_batchnorm=use_batchnorm
                ))

                # MaxPool dopo ogni residual block
                conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

                in_channels = num_filter

            self.conv_blocks = nn.Sequential(*conv_layers)

        else:
            # Standard CNN senza residual connections
            conv_layers = []
            in_channels = input_channels

            for num_filter, kernel_size in zip(num_filters, kernel_sizes):
                # Conv2d
                conv_layers.append(nn.Conv2d(
                    in_channels,
                    num_filter,
                    kernel_size=kernel_size,
                    padding=kernel_size//2
                ))

                # BatchNorm2d
                if use_batchnorm:
                    conv_layers.append(nn.BatchNorm2d(num_filter))

                # Activation
                conv_layers.append(nn.ReLU())

                # MaxPool2d
                conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

                # Dropout2d
                if dropout_rate > 0:
                    conv_layers.append(nn.Dropout2d(dropout_rate))

                in_channels = num_filter

            self.conv_blocks = nn.Sequential(*conv_layers)

        # Calcola dimensione output dopo conv blocks
        num_conv_blocks = len(num_filters)
        spatial_size = 128 // (2 ** num_conv_blocks)

        if use_global_avg_pool:
            self.global_pool = nn.AdaptiveAvgPool2d(1)
            flatten_size = num_filters[-1]
        else:
            self.global_pool = None
            flatten_size = num_filters[-1] * spatial_size * spatial_size

        # Fully connected layers per embedding
        fc_layers = []

        # FC hidden layer
        fc_layers.append(nn.Linear(flatten_size, fc_hidden_size))
        if use_batchnorm:
            fc_layers.append(nn.BatchNorm1d(fc_hidden_size))
        fc_layers.append(nn.ReLU())
        fc_layers.append(nn.Dropout(dropout_rate))

        # Embedding layer (penultimo layer)
        fc_layers.append(nn.Linear(fc_hidden_size, embedding_size))
        if use_batchnorm:
            fc_layers.append(nn.BatchNorm1d(embedding_size))
        fc_layers.append(nn.ReLU())

        self.fc_blocks = nn.Sequential(*fc_layers)

        # Classification head (solo per training)
        self.classifier = nn.Linear(embedding_size, num_classes)

    def forward(self, x):
        """
        Forward pass completo per training (con classificazione)

        Args:
            x: tensor [batch_size, channels, height, width]

        Returns:
            logits: tensor [batch_size, num_classes]
        """
        # Conv blocks
        x = self.conv_blocks(x)

        # Flatten o Global Average Pooling
        if self.use_global_avg_pool:
            x = self.global_pool(x)
            x = x.view(x.size(0), -1)
        else:
            x = x.view(x.size(0), -1)

        # FC blocks per embedding
        embedding = self.fc_blocks(x)

        # Classification
        logits = self.classifier(embedding)

        return logits

    def get_embedding(self, x):
        """
        Estrae solo l'embedding (per inference/confronto facce)

        Args:
            x: tensor [batch_size, channels, height, width]

        Returns:
            embedding: tensor [batch_size, embedding_size]
        """
        # Conv blocks
        x = self.conv_blocks(x)

        # Flatten o Global Average Pooling
        if self.use_global_avg_pool:
            x = self.global_pool(x)
            x = x.view(x.size(0), -1)
        else:
            x = x.view(x.size(0), -1)

        # FC blocks (senza classification head)
        embedding = self.fc_blocks(x)

        return embedding


# Creazione del Modello con Iperparametri

In [ ]:
# Funzione helper per creare il modello con configurazioni diverse
def create_cnn_model(config):
    """
    Factory function per creare CNN con diverse configurazioni

    Args:
        config: dict con parametri del modello

    Returns:
        model: istanza di FaceEmbeddingCNN
    """
    return FaceEmbeddingCNN(
        input_channels=config.get('input_channels', 3),
        num_filters=config.get('num_filters', [32, 64, 128]),
        kernel_sizes=config.get('kernel_sizes', [3, 3, 3]),
        fc_hidden_size=config.get('fc_hidden_size', 512),
        embedding_size=config.get('embedding_size', 128),
        num_classes=config.get('num_classes', 5749),
        dropout_rate=config.get('dropout_rate', 0.5),
        use_batchnorm=config.get('use_batchnorm', True),
        use_global_avg_pool=config.get('use_global_avg_pool', False),
        use_residual=config.get('use_residual', False)
    )

# Training Loop

## Support Training Functions

In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    Training per una singola epoca

    Args:
        model: modello da addestrare
        train_loader: DataLoader training set
        criterion: loss function
        optimizer: ottimizzatore
        device: device (cuda/cpu)

    Returns:
        avg_loss, accuracy
    """
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc="Training")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistiche
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

    avg_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy


def validate(model, test_loader, criterion, device):
    """
    Validazione sul test set

    Args:
        model: modello da validare
        test_loader: DataLoader test set
        criterion: loss function
        device: device (cuda/cpu)

    Returns:
        avg_loss, accuracy
    """
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Validation")
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })

    avg_loss = running_loss / len(test_loader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy


def save_checkpoint(model, optimizer, config, epoch, val_acc, filepath):
    """
    Salva checkpoint del modello

    Args:
        model: modello da salvare
        optimizer: ottimizzatore
        config: configurazione modello
        epoch: epoca corrente
        val_acc: accuracy validazione
        filepath: path dove salvare
    """
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'config': config,
        'epoch': epoch,
        'val_accuracy': val_acc
    }
    torch.save(checkpoint, filepath)
    print(f"Checkpoint salvato: {filepath}")


def load_checkpoint(filepath, model, optimizer=None):
    """
    Carica checkpoint del modello

    Args:
        filepath: path del checkpoint
        model: modello dove caricare i pesi
        optimizer: (opzionale) ottimizzatore dove caricare lo stato

    Returns:
        config, epoch, val_accuracy
    """
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    print(f"Checkpoint caricato: {filepath}")
    print(f"  Epoch: {checkpoint['epoch']}")
    print(f"  Val Accuracy: {checkpoint['val_accuracy']:.2f}%")

    return checkpoint['config'], checkpoint['epoch'], checkpoint['val_accuracy']


## Training Function

In [ ]:
def train_model(model, config, train_loader, test_loader,
                num_epochs=10, learning_rate=1e-3,
                checkpoint_dir='./checkpoints', device=None):
    """
    Training completo del modello

    Args:
        model: modello da addestrare
        config: configurazione modello (per salvataggio)
        train_loader: DataLoader training
        test_loader: DataLoader validation
        num_epochs: numero epoche
        learning_rate: learning rate iniziale
        checkpoint_dir: directory per salvare checkpoints
        device: device (cuda/cpu), se None rileva automaticamente

    Returns:
        model, training_history
    """
    # Device setup
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = model.to(device)

    # Loss e optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2
    )

    # Creazione directory checkpoints
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    best_val_acc = 0.0

    print("\n" + "="*60)
    print("INIZIO TRAINING")
    print("="*60 + "\n")

    #ciclo di training: 1 ciclo 1 epoca, 1 trainign, 1 validazione, 1 update scheduler
    #salva SE é il migliore
    #salva l'ultimo modello epr sicurezza
    #Qui possibilitá di aggiungere Early Stopping
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 60)

        # Training
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device
        )

        # Validation
        val_loss, val_acc = validate(
            model, test_loader, criterion, device
        )

        # Update scheduler
        scheduler.step(val_acc)

        # Salva storia
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Print epoch summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        # Salva best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
            save_checkpoint(model, optimizer, config, epoch+1, val_acc, best_model_path)
            print(f"  *** Nuovo best model! Accuracy: {val_acc:.2f}% ***")

        # Salva ultimo checkpoint
        last_checkpoint_path = os.path.join(checkpoint_dir, 'last_checkpoint.pth')
        save_checkpoint(model, optimizer, config, epoch+1, val_acc, last_checkpoint_path)

    print("\n" + "="*60)
    print("TRAINING COMPLETATO")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    print("="*60 + "\n")

    return model, history

## Plot history

In [ ]:
def plot_training_history(history, save_path=None):
    """
    Plotta loss e accuracy durante training

    Args:
        history: dizionario con 'train_loss', 'train_acc', 'val_loss', 'val_acc'
        save_path: (opzionale) path dove salvare il grafico
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    epochs = range(1, len(history['train_loss']) + 1)

    # Plot Loss
    ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r-o', label='Val Loss', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)

    # Plot Accuracy
    ax2.plot(epochs, history['train_acc'], 'b-o', label='Train Accuracy', linewidth=2)
    ax2.plot(epochs, history['val_acc'], 'r-o', label='Val Accuracy', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy (%)', fontsize=12)
    ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Grafico salvato: {save_path}")

    plt.show()

# Utilizzo Modello

Download LFW con sklearn...
Dataset scaricato:
  Totale samples: 4324
  Dimensione immagini: (125, 94, 3)
  Num classes: 158

DataLoaders creati:
  Train samples: 3459
  Test samples: 865
  Batch size: 64
Using device: cuda

INIZIO TRAINING


Epoch 1/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.56it/s, loss=4.6632, acc=12.60%]



Epoch 1 Summary:
  Train Loss: 4.8151 | Train Acc: 6.56%
  Val Loss: 4.5608 | Val Acc: 12.60%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 12.60% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 2/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 17.69it/s, loss=4.6663, acc=13.18%]



Epoch 2 Summary:
  Train Loss: 4.5348 | Train Acc: 11.36%
  Val Loss: 4.4082 | Val Acc: 13.18%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 13.18% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 3/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 17.27it/s, loss=4.6405, acc=13.64%]



Epoch 3 Summary:
  Train Loss: 4.4892 | Train Acc: 12.29%
  Val Loss: 4.3129 | Val Acc: 13.64%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 13.64% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 4/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 17.56it/s, loss=4.5658, acc=15.26%]



Epoch 4 Summary:
  Train Loss: 4.4259 | Train Acc: 12.63%
  Val Loss: 4.2305 | Val Acc: 15.26%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 15.26% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 5/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.26it/s, loss=4.4396, acc=16.65%]



Epoch 5 Summary:
  Train Loss: 4.3169 | Train Acc: 13.88%
  Val Loss: 4.1099 | Val Acc: 16.65%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 16.65% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 6/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.75it/s, loss=4.3636, acc=17.11%]



Epoch 6 Summary:
  Train Loss: 4.2121 | Train Acc: 14.95%
  Val Loss: 4.0100 | Val Acc: 17.11%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 17.11% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 7/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 17.00it/s, loss=4.2463, acc=18.96%]



Epoch 7 Summary:
  Train Loss: 4.0836 | Train Acc: 16.07%
  Val Loss: 3.8494 | Val Acc: 18.96%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 18.96% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 8/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.64it/s, loss=4.1305, acc=20.00%]



Epoch 8 Summary:
  Train Loss: 3.9729 | Train Acc: 17.58%
  Val Loss: 3.7453 | Val Acc: 20.00%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 20.00% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 9/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.81it/s, loss=3.9702, acc=22.31%]



Epoch 9 Summary:
  Train Loss: 3.8647 | Train Acc: 19.57%
  Val Loss: 3.6026 | Val Acc: 22.31%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 22.31% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

Epoch 10/10
------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.70it/s, loss=3.8387, acc=23.47%]



Epoch 10 Summary:
  Train Loss: 3.7482 | Train Acc: 20.53%
  Val Loss: 3.4840 | Val Acc: 23.47%
Checkpoint salvato: ./checkpoints/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 23.47% ***
Checkpoint salvato: ./checkpoints/cnn_baseline/last_checkpoint.pth

TRAINING COMPLETATO
Best Validation Accuracy: 23.47%

Grafico salvato: ./checkpoints/cnn_baseline/training_history.png


In [ ]:
# Configurazione
config = {
    'input_channels': 3,
    'num_filters': [32, 64, 128],
    'kernel_sizes': [3, 3, 3],
    'fc_hidden_size': 512,
    'embedding_size': 128,
    'num_classes': 5749,
    'dropout_rate': 0.5,
    'use_batchnorm': True,
    'use_global_avg_pool': False
}

# Carica dataset
train_loader, test_loader, num_classes = load_lfw_dataset(
    data_dir='./content/data',
    img_size=128,
    batch_size=64,
    augment=True
)

# Aggiorna numero classi
config['num_classes'] = num_classes

# Crea modello
model = create_cnn_model(config)

# Training
model, history = train_model(
    model=model,
    config=config,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=10,
    learning_rate=1e-3,
    checkpoint_dir='./checkpoints/cnn_baseline',
    device=None  # auto-detect
)

# Plot risultati
plot_training_history(history, save_path='./checkpoints/cnn_baseline/training_history.png')

# Different Configuration
Dato il funzionamento del codice --> cicliamo su diverse configurazioni per vedere quale é la migliore  come aggiungedo best practices migliora (speriamo) i risultati

## Importante: Early stopping
Dato che le configurazion i sono molte, non ha senso allenare una Rete se non miglira, quindi implemento Early Stopping

In [ ]:
class EarlyStopping:
    """
    Early stopping per fermare il training quando validation non migliora
    """
    def __init__(self, patience=3, min_delta=0.001, verbose=True):
        """
        Args:
            patience: quante epoche aspettare senza miglioramento
            min_delta: minimo miglioramento considerato significativo
            verbose: stampa messaggi
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_metric):
        """
        Args:
            val_metric: metrica di validazione (es. accuracy)

        Returns:
            True se deve fermarsi, False altrimenti
        """
        score = val_metric

        if self.best_score is None:
            self.best_score = score
            return False

        if score < self.best_score + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"  EarlyStopping counter: {self.counter}/{self.patience}")

            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"  Early stopping triggered!")
                return True
        else:
            self.best_score = score
            self.counter = 0

        return False

## Configruazioni utilizzate
Configurazioni diverse dalla più semplice alla più ottimizzata

In [ ]:
def get_cnn_configs():
    """
    Configurazioni CNN dalla più semplice alla più ottimizzata

    Returns:
        dict con tutte le configurazioni
    """
    configs = {
        # 1. Baseline CNN - configurazione base
        'cnn_baseline': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64, 128],  # 3 conv blocks
                'kernel_sizes': [3, 3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.5,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 15,
                'optimizer': 'adam',
                'weight_decay': 0.0,
                'label_smoothing': 0.0,
                'early_stopping_patience': 6
            }
        },

        # 2. Shallow CNN - meno layer convoluzionali
        'cnn_shallow': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64],  # Solo 2 conv blocks
                'kernel_sizes': [3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 12,
                'optimizer': 'adam',
                'weight_decay': 0.0,
                'label_smoothing': 0.0,
                'early_stopping_patience': 5
            }
        },

        # 3. Deep CNN - più layer convoluzionali
        'cnn_deep': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64, 128, 256],  # 4 conv blocks
                'kernel_sizes': [3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 5e-4,  # LR più basso per deep
                'batch_size': 32,  # Batch più piccolo
                'num_epochs': 20,
                'optimizer': 'adam',
                'weight_decay': 1e-5,
                'label_smoothing': 0.05,
                'early_stopping_patience': 7
            }
        },

        # 4. Large Kernels - kernel più grandi
        'cnn_large_kernels': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64, 128],
                'kernel_sizes': [5, 5, 5],  # Kernel 5x5
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.5,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 15,
                'optimizer': 'adam',
                'weight_decay': 0.0,
                'label_smoothing': 0.0,
                'early_stopping_patience': 6
            }
        },

        # 5. Global Average Pooling - riduce parametri
        'cnn_gap': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 128, 256],
                'kernel_sizes': [3, 3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': True  # Usa GAP
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 15,
                'optimizer': 'adam',
                'weight_decay': 1e-5,
                'label_smoothing': 0.0,
                'early_stopping_patience': 6
            }
        },

        # 6. Optimized CNN - best practices
        'cnn_optimized': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 128, 256, 512],  # Progressione standard
                'kernel_sizes': [3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.3,
                'use_batchnorm': True,
                'use_global_avg_pool': True,  # GAP per ridurre parametri
                'use_residual': False
            },
            'training_config': {
                'learning_rate': 5e-4,
                'batch_size': 64,
                'num_epochs': 25,
                'optimizer': 'adam',
                'weight_decay': 1e-4,  # L2 reg
                'label_smoothing': 0.1,  # Label smoothing
                'early_stopping_patience': 8
            }
        },

        # 7. Deep Residual CNN - rete profonda con skip connections
        'cnn_deep_residual': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 64, 128, 128, 256, 256],  # 6 residual blocks
                'kernel_sizes': [3, 3, 3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.3,
                'use_batchnorm': True,
                'use_global_avg_pool': True,
                'use_residual': True  # Skip connections attive
            },
            'training_config': {
                'learning_rate': 3e-4,  # LR più basso per deep residual
                'batch_size': 32,
                'num_epochs': 30,
                'optimizer': 'adam',
                'weight_decay': 1e-4,
                'label_smoothing': 0.1,
                'early_stopping_patience': 10
            }
        }

        # # 8. Very Deep Residual CNN - rete molto profonda con skip connections
        # 'cnn_very_deep_residual': {
        #     'model_config': {
        #         'input_channels': 3,
        #         'num_filters': [64, 64, 128, 128, 256, 256],
        #         'kernel_sizes': [3, 3, 3, 3, 3, 3],
        #         'fc_hidden_size': 1024,
        #         'embedding_size': 256,
        #         'dropout_rate': 0.4,
        #         'use_batchnorm': True,
        #         'use_global_avg_pool': True,
        #         'use_residual': True  # Skip connections attive
        #     },
        #     'training_config': {
        #         'learning_rate': 1e-4,
        #         'batch_size': 32,
        #         'num_epochs': 35,
        #         'optimizer': 'adam',
        #         'weight_decay': 5e-4,
        #         'label_smoothing': 0.15,
        #         'early_stopping_patience': 12
        #     }
        # }
    }

    return configs

## Training singola istanza di config
Pressoché identico a la cella che runnava il training precedente MA adesso legge dalle config + early stopping

In [ ]:
def train_model_with_config(model_class, config_name, config, train_loader, test_loader,
                            num_classes, checkpoint_dir='./checkpoints', device=None):
    """
    Train un modello con una specifica configurazione

    Args:
        model_class: classe del modello (es. create_dnn_model)
        config_name: nome della configurazione
        config: dict con model_config e training_config
        train_loader: DataLoader training
        test_loader: DataLoader validation
        num_classes: numero di classi
        checkpoint_dir: directory base per checkpoints
        device: device (cuda/cpu)

    Returns:
        model, history, best_val_acc
    """
    # Setup device
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Estrai configurazioni
    model_config = config['model_config'].copy()
    model_config['num_classes'] = num_classes
    training_config = config['training_config']

    # Crea modello
    model = model_class(model_config)
    model = model.to(device)

    # Loss function (con label smoothing opzionale)
    criterion = nn.CrossEntropyLoss(label_smoothing=training_config['label_smoothing'])

    # Optimizer
    if training_config['optimizer'] == 'adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=training_config['learning_rate'],
            weight_decay=training_config['weight_decay']
        )
    elif training_config['optimizer'] == 'rmsprop':
        optimizer = optim.RMSprop(
            model.parameters(),
            lr=training_config['learning_rate'],
            weight_decay=training_config['weight_decay']
        )
    else:  # sgd
        optimizer = optim.SGD(
            model.parameters(),
            lr=training_config['learning_rate'],
            momentum=0.9,
            weight_decay=training_config['weight_decay']
        )

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2
    )

    # Early stopping
    early_stopping = EarlyStopping(
        patience=training_config['early_stopping_patience'],
        verbose=True
    )

    # Directory per questa configurazione
    config_checkpoint_dir = os.path.join(checkpoint_dir, config_name)
    os.makedirs(config_checkpoint_dir, exist_ok=True)

    # Salva configurazione
    config_path = os.path.join(config_checkpoint_dir, 'config.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    best_val_acc = 0.0

    print("\n" + "="*70)
    print(f"TRAINING: {config_name}")
    print("="*70)
    print(f"Model config: {model_config}")
    print(f"Training config: {training_config}")
    print("="*70 + "\n")

    num_epochs = training_config['num_epochs']

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 70)

        # Training
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device
        )

        # Validation
        val_loss, val_acc = validate(
            model, test_loader, criterion, device
        )

        # Update scheduler
        scheduler.step(val_acc)

        # Salva storia
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # Print summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

        # Salva best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = os.path.join(config_checkpoint_dir, 'best_model.pth')
            save_checkpoint(model, optimizer, config, epoch+1, val_acc, best_model_path)
            print(f"  *** Nuovo best model! Accuracy: {val_acc:.2f}% ***")

        # Early stopping check
        if early_stopping(val_acc):
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

    # Salva ultimo checkpoint
    last_checkpoint_path = os.path.join(config_checkpoint_dir, 'last_checkpoint.pth')
    save_checkpoint(model, optimizer, config, epoch+1, val_acc, last_checkpoint_path)

    # Salva history
    history_path = os.path.join(config_checkpoint_dir, 'history.json')
    with open(history_path, 'w') as f:
        json.dump(history, f, indent=4)

    print("\n" + "="*70)
    print(f"TRAINING COMPLETATO: {config_name}")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    print("="*70 + "\n")

    return model, history, best_val_acc

## Training di tutte le diverse Config

In [ ]:
def train_multiple_configs(model_class, train_loader, test_loader, num_classes,
                          checkpoint_dir='./checkpoints', device=None):
    """
    Train modelli con tutte le configurazioni predefinite

    Args:
        model_class: classe del modello
        train_loader: DataLoader training
        test_loader: DataLoader validation
        num_classes: numero di classi
        checkpoint_dir: directory per checkpoints
        device: device (cuda/cpu)

    Returns:
        results: dict con risultati di ogni config
    """
    configs = get_cnn_configs()
    results = {}

    for config_name, config in configs.items():
        print(f"\n{'='*70}")
        print(f"Starting training: {config_name}")
        print(f"{'='*70}\n")

        try:
            model, history, best_val_acc = train_model_with_config(
                model_class=model_class,
                config_name=config_name,
                config=config,
                train_loader=train_loader,
                test_loader=test_loader,
                num_classes=num_classes,
                checkpoint_dir=checkpoint_dir,
                device=device
            )

            results[config_name] = {
                'best_val_acc': best_val_acc,
                'history': history,
                'config': config
            }

        except Exception as e:
            print(f"\nERROR training {config_name}: {e}")
            results[config_name] = {'error': str(e)}

    # Salva summary
    summary_path = os.path.join(checkpoint_dir, 'training_summary.json')
    summary = {
        name: {'best_val_acc': res.get('best_val_acc', 0)}
        for name, res in results.items()
    }
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=4)

    # Stampa summary
    print("\n" + "="*70)
    print("TRAINING SUMMARY - All Configurations")
    print("="*70)
    for name, res in results.items():
        if 'best_val_acc' in res:
            print(f"{name:20s}: {res['best_val_acc']:.2f}%")
        else:
            print(f"{name:20s}: ERROR")
    print("="*70 + "\n")

    return results

# Training Multiple Config

In [ ]:
train_loader, test_loader, num_classes = load_lfw_dataset(
    data_dir='./content/data',
    img_size=128,
    batch_size=64,
    augment=True
)

# Train tutte le configurazioni
results = train_multiple_configs(
    model_class=create_cnn_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_classes=num_classes,
    checkpoint_dir='./checkpoints/cnn_experiments',
    device=None
)

Download LFW con sklearn...
Dataset scaricato:
  Totale samples: 4324
  Dimensione immagini: (125, 94, 3)
  Num classes: 158

DataLoaders creati:
  Train samples: 3459
  Test samples: 865
  Batch size: 64

Starting training: cnn_baseline


TRAINING: cnn_baseline
Model config: {'input_channels': 3, 'num_filters': [32, 64, 128], 'kernel_sizes': [3, 3, 3], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.5, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 158}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 15, 'optimizer': 'adam', 'weight_decay': 0.0, 'label_smoothing': 0.0, 'early_stopping_patience': 6}


Epoch 1/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.40it/s, loss=4.8359, acc=12.49%]



Epoch 1 Summary:
  Train Loss: 4.8510 | Train Acc: 5.84%
  Val Loss: 4.5271 | Val Acc: 12.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 12.49% ***

Epoch 2/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.13it/s, loss=4.7083, acc=13.06%]



Epoch 2 Summary:
  Train Loss: 4.5940 | Train Acc: 11.54%
  Val Loss: 4.4280 | Val Acc: 13.06%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 13.06% ***

Epoch 3/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.26it/s, loss=4.7106, acc=13.64%]



Epoch 3 Summary:
  Train Loss: 4.4701 | Train Acc: 12.23%
  Val Loss: 4.3276 | Val Acc: 13.64%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 13.64% ***

Epoch 4/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.29it/s, loss=4.5287, acc=15.03%]



Epoch 4 Summary:
  Train Loss: 4.4109 | Train Acc: 12.95%
  Val Loss: 4.2092 | Val Acc: 15.03%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 15.03% ***

Epoch 5/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.74it/s, loss=4.5156, acc=15.95%]



Epoch 5 Summary:
  Train Loss: 4.2944 | Train Acc: 13.59%
  Val Loss: 4.1113 | Val Acc: 15.95%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 15.95% ***

Epoch 6/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.31it/s, loss=4.4216, acc=17.80%]



Epoch 6 Summary:
  Train Loss: 4.1840 | Train Acc: 14.86%
  Val Loss: 3.9853 | Val Acc: 17.80%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 17.80% ***

Epoch 7/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.57it/s, loss=4.3307, acc=18.73%]



Epoch 7 Summary:
  Train Loss: 4.1058 | Train Acc: 15.78%
  Val Loss: 3.9233 | Val Acc: 18.73%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 18.73% ***

Epoch 8/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.46it/s, loss=4.1757, acc=20.23%]



Epoch 8 Summary:
  Train Loss: 3.9873 | Train Acc: 17.43%
  Val Loss: 3.7439 | Val Acc: 20.23%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 20.23% ***

Epoch 9/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.82it/s, loss=4.0082, acc=20.92%]



Epoch 9 Summary:
  Train Loss: 3.8572 | Train Acc: 18.68%
  Val Loss: 3.6379 | Val Acc: 20.92%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 20.92% ***

Epoch 10/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.34it/s, loss=3.8985, acc=23.47%]



Epoch 10 Summary:
  Train Loss: 3.7208 | Train Acc: 20.41%
  Val Loss: 3.4913 | Val Acc: 23.47%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 23.47% ***

Epoch 11/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.70it/s, loss=3.6898, acc=26.94%]



Epoch 11 Summary:
  Train Loss: 3.5958 | Train Acc: 21.74%
  Val Loss: 3.3621 | Val Acc: 26.94%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 26.94% ***

Epoch 12/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.75it/s, loss=3.6179, acc=29.25%]



Epoch 12 Summary:
  Train Loss: 3.5015 | Train Acc: 23.56%
  Val Loss: 3.2540 | Val Acc: 29.25%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 29.25% ***

Epoch 13/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.00it/s, loss=3.4052, acc=31.79%]



Epoch 13 Summary:
  Train Loss: 3.4231 | Train Acc: 24.83%
  Val Loss: 3.1307 | Val Acc: 31.79%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 31.79% ***

Epoch 14/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.02it/s, loss=3.2447, acc=32.49%]



Epoch 14 Summary:
  Train Loss: 3.2323 | Train Acc: 27.72%
  Val Loss: 3.0226 | Val Acc: 32.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 32.49% ***

Epoch 15/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.62it/s, loss=3.1154, acc=35.38%]



Epoch 15 Summary:
  Train Loss: 3.1491 | Train Acc: 28.82%
  Val Loss: 2.9247 | Val Acc: 35.38%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 35.38% ***
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_baseline/last_checkpoint.pth

TRAINING COMPLETATO: cnn_baseline
Best Validation Accuracy: 35.38%


Starting training: cnn_shallow


TRAINING: cnn_shallow
Model config: {'input_channels': 3, 'num_filters': [32, 64], 'kernel_sizes': [3, 3], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.4, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 158}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 12, 'optimizer': 'adam', 'weight_decay': 0.0, 'label_smoothing': 0.0, 'early_stopping_patience': 5}


Epoch 1/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.73it/s, loss=4.7356, acc=14.34%]



Epoch 1 Summary:
  Train Loss: 4.7959 | Train Acc: 7.08%
  Val Loss: 4.5105 | Val Acc: 14.34%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 14.34% ***

Epoch 2/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.97it/s, loss=4.6117, acc=15.49%]



Epoch 2 Summary:
  Train Loss: 4.4554 | Train Acc: 12.98%
  Val Loss: 4.2468 | Val Acc: 15.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 15.49% ***

Epoch 3/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.50it/s, loss=4.5560, acc=16.18%]



Epoch 3 Summary:
  Train Loss: 4.2988 | Train Acc: 14.37%
  Val Loss: 4.0825 | Val Acc: 16.18%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 16.18% ***

Epoch 4/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.41it/s, loss=4.3429, acc=18.15%]



Epoch 4 Summary:
  Train Loss: 4.1032 | Train Acc: 15.87%
  Val Loss: 3.8827 | Val Acc: 18.15%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 18.15% ***

Epoch 5/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.77it/s, loss=4.2020, acc=20.12%]



Epoch 5 Summary:
  Train Loss: 3.9598 | Train Acc: 18.76%
  Val Loss: 3.7593 | Val Acc: 20.12%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 20.12% ***

Epoch 6/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.71it/s, loss=3.8773, acc=24.51%]



Epoch 6 Summary:
  Train Loss: 3.7683 | Train Acc: 20.82%
  Val Loss: 3.5096 | Val Acc: 24.51%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 24.51% ***

Epoch 7/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.57it/s, loss=3.7424, acc=27.40%]



Epoch 7 Summary:
  Train Loss: 3.6070 | Train Acc: 22.72%
  Val Loss: 3.3416 | Val Acc: 27.40%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 27.40% ***

Epoch 8/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.35it/s, loss=3.5555, acc=28.44%]



Epoch 8 Summary:
  Train Loss: 3.4611 | Train Acc: 25.59%
  Val Loss: 3.2213 | Val Acc: 28.44%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 28.44% ***

Epoch 9/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.58it/s, loss=3.4153, acc=30.87%]



Epoch 9 Summary:
  Train Loss: 3.2965 | Train Acc: 27.75%
  Val Loss: 3.0851 | Val Acc: 30.87%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 30.87% ***

Epoch 10/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.62it/s, loss=3.2406, acc=33.87%]



Epoch 10 Summary:
  Train Loss: 3.1791 | Train Acc: 29.58%
  Val Loss: 2.9126 | Val Acc: 33.87%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 33.87% ***

Epoch 11/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.31it/s, loss=3.1454, acc=35.49%]



Epoch 11 Summary:
  Train Loss: 3.0438 | Train Acc: 31.66%
  Val Loss: 2.7907 | Val Acc: 35.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 35.49% ***

Epoch 12/12
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  9.90it/s, loss=2.9954, acc=38.38%]



Epoch 12 Summary:
  Train Loss: 2.8981 | Train Acc: 34.32%
  Val Loss: 2.7035 | Val Acc: 38.38%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 38.38% ***
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_shallow/last_checkpoint.pth

TRAINING COMPLETATO: cnn_shallow
Best Validation Accuracy: 38.38%


Starting training: cnn_deep


TRAINING: cnn_deep
Model config: {'input_channels': 3, 'num_filters': [32, 64, 128, 256], 'kernel_sizes': [3, 3, 3, 3], 'fc_hidden_size': 1024, 'embedding_size': 256, 'dropout_rate': 0.4, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 158}
Training config: {'learning_rate': 0.0005, 'batch_size': 32, 'num_epochs': 20, 'optimizer': 'adam', 'weight_decay': 1e-05, 'label_smoothing': 0.05, 'early_stopping_patience': 7}


Epoch 1/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  9.76it/s, loss=4.8499, acc=12.49%]



Epoch 1 Summary:
  Train Loss: 4.8657 | Train Acc: 6.91%
  Val Loss: 4.5742 | Val Acc: 12.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 12.49% ***

Epoch 2/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.48it/s, loss=4.6944, acc=13.41%]



Epoch 2 Summary:
  Train Loss: 4.5460 | Train Acc: 11.36%
  Val Loss: 4.5246 | Val Acc: 13.41%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 13.41% ***

Epoch 3/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.41it/s, loss=4.7298, acc=13.41%]



Epoch 3 Summary:
  Train Loss: 4.4644 | Train Acc: 12.29%
  Val Loss: 4.3778 | Val Acc: 13.41%
  EarlyStopping counter: 1/7

Epoch 4/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.21it/s, loss=4.5322, acc=15.26%]



Epoch 4 Summary:
  Train Loss: 4.3589 | Train Acc: 13.59%
  Val Loss: 4.2103 | Val Acc: 15.26%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 15.26% ***

Epoch 5/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.35it/s, loss=4.3805, acc=16.99%]



Epoch 5 Summary:
  Train Loss: 4.2044 | Train Acc: 15.52%
  Val Loss: 4.0333 | Val Acc: 16.99%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 16.99% ***

Epoch 6/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.56it/s, loss=4.2576, acc=20.00%]



Epoch 6 Summary:
  Train Loss: 4.0451 | Train Acc: 18.68%
  Val Loss: 3.8788 | Val Acc: 20.00%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 20.00% ***

Epoch 7/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.41it/s, loss=4.1682, acc=21.97%]



Epoch 7 Summary:
  Train Loss: 3.8636 | Train Acc: 20.61%
  Val Loss: 3.7503 | Val Acc: 21.97%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 21.97% ***

Epoch 8/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.51it/s, loss=3.9537, acc=25.78%]



Epoch 8 Summary:
  Train Loss: 3.7358 | Train Acc: 23.19%
  Val Loss: 3.5416 | Val Acc: 25.78%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 25.78% ***

Epoch 9/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.70it/s, loss=3.7429, acc=29.25%]



Epoch 9 Summary:
  Train Loss: 3.5708 | Train Acc: 25.47%
  Val Loss: 3.3885 | Val Acc: 29.25%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 29.25% ***

Epoch 10/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.82it/s, loss=3.5824, acc=31.21%]



Epoch 10 Summary:
  Train Loss: 3.4228 | Train Acc: 28.13%
  Val Loss: 3.2832 | Val Acc: 31.21%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 31.21% ***

Epoch 11/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.63it/s, loss=3.3953, acc=34.34%]



Epoch 11 Summary:
  Train Loss: 3.3025 | Train Acc: 30.82%
  Val Loss: 3.1187 | Val Acc: 34.34%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 34.34% ***

Epoch 12/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.35it/s, loss=3.3237, acc=35.84%]



Epoch 12 Summary:
  Train Loss: 3.1818 | Train Acc: 32.00%
  Val Loss: 3.0309 | Val Acc: 35.84%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 35.84% ***

Epoch 13/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 16.13it/s, loss=3.1352, acc=41.16%]



Epoch 13 Summary:
  Train Loss: 3.0394 | Train Acc: 35.96%
  Val Loss: 2.8713 | Val Acc: 41.16%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 41.16% ***

Epoch 14/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.69it/s, loss=2.9694, acc=42.66%]



Epoch 14 Summary:
  Train Loss: 2.9027 | Train Acc: 38.80%
  Val Loss: 2.7668 | Val Acc: 42.66%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 42.66% ***

Epoch 15/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.85it/s, loss=2.8600, acc=45.20%]



Epoch 15 Summary:
  Train Loss: 2.7867 | Train Acc: 41.08%
  Val Loss: 2.6625 | Val Acc: 45.20%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 45.20% ***

Epoch 16/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.76it/s, loss=2.7007, acc=48.55%]



Epoch 16 Summary:
  Train Loss: 2.6644 | Train Acc: 44.15%
  Val Loss: 2.5226 | Val Acc: 48.55%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 48.55% ***

Epoch 17/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.36it/s, loss=2.6035, acc=51.10%]



Epoch 17 Summary:
  Train Loss: 2.5773 | Train Acc: 45.85%
  Val Loss: 2.4180 | Val Acc: 51.10%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 51.10% ***

Epoch 18/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.91it/s, loss=2.4966, acc=53.18%]



Epoch 18 Summary:
  Train Loss: 2.4742 | Train Acc: 47.90%
  Val Loss: 2.3431 | Val Acc: 53.18%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 53.18% ***

Epoch 19/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.62it/s, loss=2.4655, acc=53.64%]



Epoch 19 Summary:
  Train Loss: 2.3744 | Train Acc: 50.27%
  Val Loss: 2.2913 | Val Acc: 53.64%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 53.64% ***

Epoch 20/20
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.67it/s, loss=2.3114, acc=54.57%]



Epoch 20 Summary:
  Train Loss: 2.3140 | Train Acc: 51.92%
  Val Loss: 2.2345 | Val Acc: 54.57%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/best_model.pth
  *** Nuovo best model! Accuracy: 54.57% ***
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep/last_checkpoint.pth

TRAINING COMPLETATO: cnn_deep
Best Validation Accuracy: 54.57%


Starting training: cnn_large_kernels


TRAINING: cnn_large_kernels
Model config: {'input_channels': 3, 'num_filters': [32, 64, 128], 'kernel_sizes': [5, 5, 5], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.5, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 158}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 15, 'optimizer': 'adam', 'weight_decay': 0.0, 'label_smoothing': 0.0, 'early_stopping_patience': 6}


Epoch 1/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.39it/s, loss=4.8262, acc=12.49%]



Epoch 1 Summary:
  Train Loss: 4.8128 | Train Acc: 8.27%
  Val Loss: 4.5153 | Val Acc: 12.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 12.49% ***

Epoch 2/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.17it/s, loss=4.8118, acc=13.64%]



Epoch 2 Summary:
  Train Loss: 4.5743 | Train Acc: 12.43%
  Val Loss: 4.4324 | Val Acc: 13.64%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 13.64% ***

Epoch 3/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.10it/s, loss=4.7862, acc=13.53%]



Epoch 3 Summary:
  Train Loss: 4.5064 | Train Acc: 12.43%
  Val Loss: 4.3928 | Val Acc: 13.53%
  EarlyStopping counter: 1/6

Epoch 4/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.37it/s, loss=4.7028, acc=14.57%]



Epoch 4 Summary:
  Train Loss: 4.4213 | Train Acc: 12.92%
  Val Loss: 4.2811 | Val Acc: 14.57%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 14.57% ***

Epoch 5/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.03it/s, loss=4.5118, acc=15.49%]



Epoch 5 Summary:
  Train Loss: 4.3326 | Train Acc: 13.33%
  Val Loss: 4.1967 | Val Acc: 15.49%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 15.49% ***

Epoch 6/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.18it/s, loss=4.4154, acc=16.07%]



Epoch 6 Summary:
  Train Loss: 4.2502 | Train Acc: 14.31%
  Val Loss: 4.0830 | Val Acc: 16.07%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 16.07% ***

Epoch 7/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.31it/s, loss=4.3557, acc=17.34%]



Epoch 7 Summary:
  Train Loss: 4.1402 | Train Acc: 15.44%
  Val Loss: 3.9956 | Val Acc: 17.34%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 17.34% ***

Epoch 8/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.38it/s, loss=4.1194, acc=18.84%]



Epoch 8 Summary:
  Train Loss: 4.0239 | Train Acc: 16.22%
  Val Loss: 3.8370 | Val Acc: 18.84%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 18.84% ***

Epoch 9/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.16it/s, loss=4.0732, acc=20.46%]



Epoch 9 Summary:
  Train Loss: 3.9025 | Train Acc: 17.87%
  Val Loss: 3.7516 | Val Acc: 20.46%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 20.46% ***

Epoch 10/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.29it/s, loss=3.9375, acc=22.08%]



Epoch 10 Summary:
  Train Loss: 3.8393 | Train Acc: 18.07%
  Val Loss: 3.6455 | Val Acc: 22.08%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 22.08% ***

Epoch 11/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.95it/s, loss=3.8379, acc=23.01%]



Epoch 11 Summary:
  Train Loss: 3.7142 | Train Acc: 19.63%
  Val Loss: 3.5456 | Val Acc: 23.01%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 23.01% ***

Epoch 12/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.38it/s, loss=3.6034, acc=24.97%]



Epoch 12 Summary:
  Train Loss: 3.6105 | Train Acc: 20.93%
  Val Loss: 3.3987 | Val Acc: 24.97%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 24.97% ***

Epoch 13/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.18it/s, loss=3.6185, acc=26.36%]



Epoch 13 Summary:
  Train Loss: 3.5107 | Train Acc: 22.95%
  Val Loss: 3.3441 | Val Acc: 26.36%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 26.36% ***

Epoch 14/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  9.06it/s, loss=3.4396, acc=28.67%]



Epoch 14 Summary:
  Train Loss: 3.4034 | Train Acc: 24.49%
  Val Loss: 3.1829 | Val Acc: 28.67%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 28.67% ***

Epoch 15/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.14it/s, loss=3.2626, acc=32.25%]



Epoch 15 Summary:
  Train Loss: 3.2814 | Train Acc: 25.70%
  Val Loss: 3.0873 | Val Acc: 32.25%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/best_model.pth
  *** Nuovo best model! Accuracy: 32.25% ***
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_large_kernels/last_checkpoint.pth

TRAINING COMPLETATO: cnn_large_kernels
Best Validation Accuracy: 32.25%


Starting training: cnn_gap


TRAINING: cnn_gap
Model config: {'input_channels': 3, 'num_filters': [64, 128, 256], 'kernel_sizes': [3, 3, 3], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.4, 'use_batchnorm': True, 'use_global_avg_pool': True, 'num_classes': 158}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 15, 'optimizer': 'adam', 'weight_decay': 1e-05, 'label_smoothing': 0.0, 'early_stopping_patience': 6}


Epoch 1/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.34it/s, loss=4.8204, acc=12.02%]



Epoch 1 Summary:
  Train Loss: 4.8360 | Train Acc: 7.29%
  Val Loss: 4.5787 | Val Acc: 12.02%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_gap/best_model.pth
  *** Nuovo best model! Accuracy: 12.02% ***

Epoch 2/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.56it/s, loss=4.8432, acc=13.06%]



Epoch 2 Summary:
  Train Loss: 4.5447 | Train Acc: 12.20%
  Val Loss: 4.4443 | Val Acc: 13.06%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_gap/best_model.pth
  *** Nuovo best model! Accuracy: 13.06% ***

Epoch 3/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.32it/s, loss=4.9202, acc=12.95%]



Epoch 3 Summary:
  Train Loss: 4.5123 | Train Acc: 12.14%
  Val Loss: 4.5003 | Val Acc: 12.95%
  EarlyStopping counter: 1/6

Epoch 4/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.54it/s, loss=4.7131, acc=12.95%]



Epoch 4 Summary:
  Train Loss: 4.4872 | Train Acc: 12.40%
  Val Loss: 4.4345 | Val Acc: 12.95%
  EarlyStopping counter: 2/6

Epoch 5/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.18it/s, loss=4.6776, acc=12.72%]



Epoch 5 Summary:
  Train Loss: 4.4856 | Train Acc: 12.06%
  Val Loss: 4.4192 | Val Acc: 12.72%
  EarlyStopping counter: 3/6

Epoch 6/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.48it/s, loss=4.6780, acc=12.83%]



Epoch 6 Summary:
  Train Loss: 4.4614 | Train Acc: 12.17%
  Val Loss: 4.4063 | Val Acc: 12.83%
  EarlyStopping counter: 4/6

Epoch 7/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.15it/s, loss=4.6601, acc=13.06%]



Epoch 7 Summary:
  Train Loss: 4.4447 | Train Acc: 12.52%
  Val Loss: 4.4115 | Val Acc: 13.06%
  EarlyStopping counter: 5/6

Epoch 8/15
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.93it/s, loss=4.6566, acc=12.95%]



Epoch 8 Summary:
  Train Loss: 4.4551 | Train Acc: 12.60%
  Val Loss: 4.4046 | Val Acc: 12.95%
  EarlyStopping counter: 6/6
  Early stopping triggered!

Early stopping at epoch 8
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_gap/last_checkpoint.pth

TRAINING COMPLETATO: cnn_gap
Best Validation Accuracy: 13.06%


Starting training: cnn_optimized


TRAINING: cnn_optimized
Model config: {'input_channels': 3, 'num_filters': [64, 128, 256, 512], 'kernel_sizes': [3, 3, 3, 3], 'fc_hidden_size': 1024, 'embedding_size': 256, 'dropout_rate': 0.3, 'use_batchnorm': True, 'use_global_avg_pool': True, 'use_residual': False, 'num_classes': 158}
Training config: {'learning_rate': 0.0005, 'batch_size': 64, 'num_epochs': 25, 'optimizer': 'adam', 'weight_decay': 0.0001, 'label_smoothing': 0.1, 'early_stopping_patience': 8}


Epoch 1/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 14.00it/s, loss=4.9203, acc=12.95%]



Epoch 1 Summary:
  Train Loss: 4.8515 | Train Acc: 8.70%
  Val Loss: 4.6012 | Val Acc: 12.95%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_optimized/best_model.pth
  *** Nuovo best model! Accuracy: 12.95% ***

Epoch 2/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.94it/s, loss=4.7980, acc=13.06%]



Epoch 2 Summary:
  Train Loss: 4.6464 | Train Acc: 12.11%
  Val Loss: 4.5396 | Val Acc: 13.06%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_optimized/best_model.pth
  *** Nuovo best model! Accuracy: 13.06% ***

Epoch 3/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.20it/s, loss=4.7880, acc=12.72%]



Epoch 3 Summary:
  Train Loss: 4.6065 | Train Acc: 12.06%
  Val Loss: 4.5311 | Val Acc: 12.72%
  EarlyStopping counter: 1/8

Epoch 4/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.11it/s, loss=4.7729, acc=13.06%]



Epoch 4 Summary:
  Train Loss: 4.5284 | Train Acc: 12.46%
  Val Loss: 4.4893 | Val Acc: 13.06%
  EarlyStopping counter: 2/8

Epoch 5/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.00it/s, loss=4.6844, acc=13.06%]



Epoch 5 Summary:
  Train Loss: 4.5321 | Train Acc: 12.63%
  Val Loss: 4.4867 | Val Acc: 13.06%
  EarlyStopping counter: 3/8

Epoch 6/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.48it/s, loss=4.7131, acc=12.83%]



Epoch 6 Summary:
  Train Loss: 4.5048 | Train Acc: 12.55%
  Val Loss: 4.4621 | Val Acc: 12.83%
  EarlyStopping counter: 4/8

Epoch 7/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.99it/s, loss=4.7212, acc=12.83%]



Epoch 7 Summary:
  Train Loss: 4.4988 | Train Acc: 12.63%
  Val Loss: 4.4600 | Val Acc: 12.83%
  EarlyStopping counter: 5/8

Epoch 8/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.67it/s, loss=4.6922, acc=12.60%]



Epoch 8 Summary:
  Train Loss: 4.4809 | Train Acc: 12.98%
  Val Loss: 4.4656 | Val Acc: 12.60%
  EarlyStopping counter: 6/8

Epoch 9/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.24it/s, loss=4.6627, acc=13.29%]



Epoch 9 Summary:
  Train Loss: 4.4618 | Train Acc: 12.92%
  Val Loss: 4.4372 | Val Acc: 13.29%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_optimized/best_model.pth
  *** Nuovo best model! Accuracy: 13.29% ***

Epoch 10/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.90it/s, loss=4.6748, acc=12.95%]



Epoch 10 Summary:
  Train Loss: 4.4484 | Train Acc: 12.78%
  Val Loss: 4.4424 | Val Acc: 12.95%
  EarlyStopping counter: 1/8

Epoch 11/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.15it/s, loss=4.6973, acc=12.95%]



Epoch 11 Summary:
  Train Loss: 4.4463 | Train Acc: 12.72%
  Val Loss: 4.4433 | Val Acc: 12.95%
  EarlyStopping counter: 2/8

Epoch 12/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.16it/s, loss=4.6970, acc=12.72%]



Epoch 12 Summary:
  Train Loss: 4.4449 | Train Acc: 12.72%
  Val Loss: 4.4406 | Val Acc: 12.72%
  EarlyStopping counter: 3/8

Epoch 13/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.59it/s, loss=4.6822, acc=12.72%]



Epoch 13 Summary:
  Train Loss: 4.4048 | Train Acc: 12.72%
  Val Loss: 4.4313 | Val Acc: 12.72%
  EarlyStopping counter: 4/8

Epoch 14/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.08it/s, loss=4.6884, acc=12.60%]



Epoch 14 Summary:
  Train Loss: 4.4279 | Train Acc: 13.07%
  Val Loss: 4.4474 | Val Acc: 12.60%
  EarlyStopping counter: 5/8

Epoch 15/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 15.01it/s, loss=4.6975, acc=12.83%]



Epoch 15 Summary:
  Train Loss: 4.4072 | Train Acc: 13.04%
  Val Loss: 4.4350 | Val Acc: 12.83%
  EarlyStopping counter: 6/8

Epoch 16/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.27it/s, loss=4.7042, acc=12.83%]



Epoch 16 Summary:
  Train Loss: 4.4283 | Train Acc: 12.66%
  Val Loss: 4.4291 | Val Acc: 12.83%
  EarlyStopping counter: 7/8

Epoch 17/25
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.23it/s, loss=4.6975, acc=12.72%]



Epoch 17 Summary:
  Train Loss: 4.4104 | Train Acc: 12.72%
  Val Loss: 4.4409 | Val Acc: 12.72%
  EarlyStopping counter: 8/8
  Early stopping triggered!

Early stopping at epoch 17
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_optimized/last_checkpoint.pth

TRAINING COMPLETATO: cnn_optimized
Best Validation Accuracy: 13.29%


Starting training: cnn_deep_residual


TRAINING: cnn_deep_residual
Model config: {'input_channels': 3, 'num_filters': [64, 64, 128, 128, 256, 256], 'kernel_sizes': [3, 3, 3, 3, 3, 3], 'fc_hidden_size': 1024, 'embedding_size': 256, 'dropout_rate': 0.3, 'use_batchnorm': True, 'use_global_avg_pool': True, 'use_residual': True, 'num_classes': 158}
Training config: {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 30, 'optimizer': 'adam', 'weight_decay': 0.0001, 'label_smoothing': 0.1, 'early_stopping_patience': 10}


Epoch 1/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.15it/s, loss=4.8624, acc=5.55%]



Epoch 1 Summary:
  Train Loss: 4.9164 | Train Acc: 7.46%
  Val Loss: 4.7858 | Val Acc: 5.55%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 5.55% ***

Epoch 2/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.82it/s, loss=4.8517, acc=12.14%]



Epoch 2 Summary:
  Train Loss: 4.7186 | Train Acc: 11.04%
  Val Loss: 4.6323 | Val Acc: 12.14%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 12.14% ***

Epoch 3/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.23it/s, loss=4.7758, acc=12.25%]



Epoch 3 Summary:
  Train Loss: 4.6548 | Train Acc: 11.59%
  Val Loss: 4.5957 | Val Acc: 12.25%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 12.25% ***

Epoch 4/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.82it/s, loss=4.7787, acc=13.18%]



Epoch 4 Summary:
  Train Loss: 4.6107 | Train Acc: 12.43%
  Val Loss: 4.5631 | Val Acc: 13.18%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 13.18% ***

Epoch 5/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.58it/s, loss=4.7638, acc=12.72%]



Epoch 5 Summary:
  Train Loss: 4.5929 | Train Acc: 12.32%
  Val Loss: 4.5304 | Val Acc: 12.72%
  EarlyStopping counter: 1/10

Epoch 6/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.52it/s, loss=4.6888, acc=12.95%]



Epoch 6 Summary:
  Train Loss: 4.5885 | Train Acc: 12.63%
  Val Loss: 4.5012 | Val Acc: 12.95%
  EarlyStopping counter: 2/10

Epoch 7/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.97it/s, loss=4.6531, acc=12.95%]



Epoch 7 Summary:
  Train Loss: 4.5752 | Train Acc: 12.89%
  Val Loss: 4.4702 | Val Acc: 12.95%
  EarlyStopping counter: 3/10

Epoch 8/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.62it/s, loss=4.6463, acc=12.95%]



Epoch 8 Summary:
  Train Loss: 4.5407 | Train Acc: 12.63%
  Val Loss: 4.4691 | Val Acc: 12.95%
  EarlyStopping counter: 4/10

Epoch 9/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.63it/s, loss=4.7010, acc=12.83%]



Epoch 9 Summary:
  Train Loss: 4.5329 | Train Acc: 13.10%
  Val Loss: 4.4712 | Val Acc: 12.83%
  EarlyStopping counter: 5/10

Epoch 10/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.88it/s, loss=4.6645, acc=12.83%]



Epoch 10 Summary:
  Train Loss: 4.5255 | Train Acc: 12.69%
  Val Loss: 4.4575 | Val Acc: 12.83%
  EarlyStopping counter: 6/10

Epoch 11/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.90it/s, loss=4.6337, acc=13.06%]



Epoch 11 Summary:
  Train Loss: 4.4781 | Train Acc: 12.37%
  Val Loss: 4.4311 | Val Acc: 13.06%
  EarlyStopping counter: 7/10

Epoch 12/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.76it/s, loss=4.6139, acc=13.06%]



Epoch 12 Summary:
  Train Loss: 4.4449 | Train Acc: 12.86%
  Val Loss: 4.4097 | Val Acc: 13.06%
  EarlyStopping counter: 8/10

Epoch 13/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.92it/s, loss=4.6530, acc=13.29%]



Epoch 13 Summary:
  Train Loss: 4.4583 | Train Acc: 12.43%
  Val Loss: 4.4235 | Val Acc: 13.29%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 13.29% ***

Epoch 14/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.54it/s, loss=4.6520, acc=13.06%]



Epoch 14 Summary:
  Train Loss: 4.4386 | Train Acc: 12.75%
  Val Loss: 4.4201 | Val Acc: 13.06%
  EarlyStopping counter: 1/10

Epoch 15/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 11.76it/s, loss=4.6350, acc=13.18%]



Epoch 15 Summary:
  Train Loss: 4.4475 | Train Acc: 13.24%
  Val Loss: 4.4120 | Val Acc: 13.18%
  EarlyStopping counter: 2/10

Epoch 16/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.03it/s, loss=4.6400, acc=13.29%]



Epoch 16 Summary:
  Train Loss: 4.4257 | Train Acc: 12.81%
  Val Loss: 4.4070 | Val Acc: 13.29%
  EarlyStopping counter: 3/10

Epoch 17/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 12.03it/s, loss=4.5927, acc=13.18%]



Epoch 17 Summary:
  Train Loss: 4.4087 | Train Acc: 12.75%
  Val Loss: 4.3884 | Val Acc: 13.18%
  EarlyStopping counter: 4/10

Epoch 18/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.72it/s, loss=4.6000, acc=13.41%]



Epoch 18 Summary:
  Train Loss: 4.4036 | Train Acc: 12.72%
  Val Loss: 4.3817 | Val Acc: 13.41%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 13.41% ***

Epoch 19/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00, 10.33it/s, loss=4.6234, acc=13.41%]



Epoch 19 Summary:
  Train Loss: 4.4002 | Train Acc: 12.66%
  Val Loss: 4.3846 | Val Acc: 13.41%
  EarlyStopping counter: 1/10

Epoch 20/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  9.48it/s, loss=4.6124, acc=13.18%]



Epoch 20 Summary:
  Train Loss: 4.3615 | Train Acc: 13.07%
  Val Loss: 4.3800 | Val Acc: 13.18%
  EarlyStopping counter: 2/10

Epoch 21/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  9.16it/s, loss=4.6149, acc=13.06%]



Epoch 21 Summary:
  Train Loss: 4.3728 | Train Acc: 12.86%
  Val Loss: 4.3864 | Val Acc: 13.06%
  EarlyStopping counter: 3/10

Epoch 22/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.91it/s, loss=4.6253, acc=13.18%]



Epoch 22 Summary:
  Train Loss: 4.3791 | Train Acc: 13.39%
  Val Loss: 4.3810 | Val Acc: 13.18%
  EarlyStopping counter: 4/10

Epoch 23/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.48it/s, loss=4.5962, acc=13.29%]



Epoch 23 Summary:
  Train Loss: 4.3745 | Train Acc: 13.62%
  Val Loss: 4.3529 | Val Acc: 13.29%
  EarlyStopping counter: 5/10

Epoch 24/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.64it/s, loss=4.5953, acc=13.53%]



Epoch 24 Summary:
  Train Loss: 4.3654 | Train Acc: 13.41%
  Val Loss: 4.3575 | Val Acc: 13.53%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 13.53% ***

Epoch 25/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.62it/s, loss=4.5934, acc=13.76%]



Epoch 25 Summary:
  Train Loss: 4.3645 | Train Acc: 13.53%
  Val Loss: 4.3305 | Val Acc: 13.76%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 13.76% ***

Epoch 26/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.79it/s, loss=4.6194, acc=13.53%]



Epoch 26 Summary:
  Train Loss: 4.3542 | Train Acc: 13.27%
  Val Loss: 4.3543 | Val Acc: 13.53%
  EarlyStopping counter: 1/10

Epoch 27/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.58it/s, loss=4.6217, acc=13.64%]



Epoch 27 Summary:
  Train Loss: 4.3781 | Train Acc: 13.04%
  Val Loss: 4.3566 | Val Acc: 13.64%
  EarlyStopping counter: 2/10

Epoch 28/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.85it/s, loss=4.6016, acc=13.53%]



Epoch 28 Summary:
  Train Loss: 4.3541 | Train Acc: 13.47%
  Val Loss: 4.3452 | Val Acc: 13.53%
  EarlyStopping counter: 3/10

Epoch 29/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.64it/s, loss=4.5890, acc=14.10%]



Epoch 29 Summary:
  Train Loss: 4.3358 | Train Acc: 13.65%
  Val Loss: 4.3259 | Val Acc: 14.10%
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/best_model.pth
  *** Nuovo best model! Accuracy: 14.10% ***

Epoch 30/30
----------------------------------------------------------------------


Validation: 100%|██████████| 14/14 [00:01<00:00,  8.59it/s, loss=4.5923, acc=13.99%]



Epoch 30 Summary:
  Train Loss: 4.3444 | Train Acc: 13.67%
  Val Loss: 4.3275 | Val Acc: 13.99%
  EarlyStopping counter: 1/10
Checkpoint salvato: ./checkpoints/cnn_experiments/cnn_deep_residual/last_checkpoint.pth

TRAINING COMPLETATO: cnn_deep_residual
Best Validation Accuracy: 14.10%


Starting training: cnn_very_deep_residual


TRAINING: cnn_very_deep_residual
Model config: {'input_channels': 3, 'num_filters': [64, 64, 128, 128, 256, 256, 512, 512], 'kernel_sizes': [3, 3, 3, 3, 3, 3, 3, 3], 'fc_hidden_size': 2048, 'embedding_size': 512, 'dropout_rate': 0.4, 'use_batchnorm': True, 'use_global_avg_pool': True, 'use_residual': True, 'num_classes': 158}
Training config: {'learning_rate': 0.0001, 'batch_size': 16, 'num_epochs': 35, 'optimizer': 'adam', 'weight_decay': 0.0005, 'label_smoothing': 0.15, 'early_stopping_patience': 12}


Epoch 1/35
----------------------------------------------------------------------


Training:   0%|          | 0/55 [00:00<?, ?it/s]


ERROR training cnn_very_deep_residual: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small

TRAINING SUMMARY - All Configurations
cnn_baseline        : 35.38%
cnn_shallow         : 38.38%
cnn_deep            : 54.57%
cnn_large_kernels   : 32.25%
cnn_gap             : 13.06%
cnn_optimized       : 13.29%
cnn_deep_residual   : 14.10%
cnn_very_deep_residual: ERROR



# Riassunto Modelli

In [ ]:
def plot_overfitting_analysis(checkpoint_dir='./checkpoints/cnn_experiments', save_path=None):
    """
    Analizza e plotta l'overfitting: confronta train vs val accuracy con gap evidenziato

    Args:
        checkpoint_dir: directory con le sottocartelle delle config
        save_path: (opzionale) path dove salvare il grafico
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()

    config_names = sorted([d for d in os.listdir(checkpoint_dir)
                          if os.path.isdir(os.path.join(checkpoint_dir, d))])

    overfitting_data = []

    for idx, config_name in enumerate(config_names[:6]):  # Max 6 plots
        history_path = os.path.join(checkpoint_dir, config_name, 'history.json')

        if not os.path.exists(history_path):
            continue

        with open(history_path, 'r') as f:
            history = json.load(f)

        epochs = range(1, len(history['train_acc']) + 1)
        train_acc = history['train_acc']
        val_acc = history['val_acc']

        # Calcola gap (overfitting indicator)
        final_train = train_acc[-1]
        final_val = val_acc[-1]
        gap = final_train - final_val

        overfitting_data.append({
            'config': config_name,
            'train_acc': final_train,
            'val_acc': final_val,
            'gap': gap
        })

        # Plot train vs val
        ax = axes[idx]
        ax.plot(epochs, train_acc, 'b-o', label='Train Acc', linewidth=2, markersize=4)
        ax.plot(epochs, val_acc, 'r-s', label='Val Acc', linewidth=2, markersize=4)

        # Evidenzia gap finale
        ax.axhline(y=final_train, color='b', linestyle='--', alpha=0.3)
        ax.axhline(y=final_val, color='r', linestyle='--', alpha=0.3)
        ax.fill_between(epochs, final_train, final_val, alpha=0.2, color='orange')

        # Aggiungi testo con gap
        gap_text = f'Gap: {gap:.1f}%'
        if gap > 15:
            gap_text += ' (OVERFITTING!)'
            color = 'red'
        elif gap > 5:
            gap_text += ' (Moderate)'
            color = 'orange'
        else:
            gap_text += ' (Good)'
            color = 'green'

        ax.text(0.5, 0.95, gap_text, transform=ax.transAxes,
               fontsize=11, fontweight='bold', color=color,
               ha='center', va='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        ax.set_xlabel('Epoch', fontsize=10)
        ax.set_ylabel('Accuracy (%)', fontsize=10)
        ax.set_title(f'{config_name}\nTrain: {final_train:.1f}% | Val: {final_val:.1f}%',
                    fontsize=11, fontweight='bold')
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)

    # Nascondi assi non usati
    for idx in range(len(config_names), 6):
        axes[idx].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Grafico overfitting salvato: {save_path}")

    plt.show()

    # Stampa tabella overfitting
    print("\n" + "="*80)
    print("OVERFITTING ANALYSIS")
    print("="*80)
    print(f"{'Config':<25} | {'Train Acc':>10} | {'Val Acc':>10} | {'Gap':>10} | {'Status':<15}")
    print("-"*80)

    for data in sorted(overfitting_data, key=lambda x: x['gap'], reverse=True):
        if data['gap'] > 15:
            status = 'OVERFITTING!'
        elif data['gap'] > 5:
            status = 'Moderate'
        else:
            status = 'Good'

        print(f"{data['config']:<25} | {data['train_acc']:>9.2f}% | {data['val_acc']:>9.2f}% | "
              f"{data['gap']:>9.2f}% | {status:<15}")

    print("="*80)
    print("Note: Gap = Train Acc - Val Acc")
    print("  Gap > 15% = Severe overfitting")
    print("  Gap 5-15% = Moderate overfitting")
    print("  Gap < 5%  = Good generalization")
    print("="*80 + "\n")

In [ ]:
def print_models_summary(model_class, checkpoint_dir='./checkpoints/cnn_experiments'):
    """
    Stampa tabella comparativa con stats di tutti i modelli trainati

    Args:
        model_class: classe del modello (es. create_dnn_model)
        checkpoint_dir: directory con le sottocartelle delle config
    """
    print("\n" + "="*90)
    print("SUMMARY - All Trained Models")
    print("="*90)
    print(f"{'Config':<20} | {'Parameters':>15} | {'Best Val Acc':>13} | {'Epochs':>7}")
    print("-"*90)

    results = []

    for config_name in sorted(os.listdir(checkpoint_dir)):
        config_path = os.path.join(checkpoint_dir, config_name)

        if not os.path.isdir(config_path):
            continue

        # Carica best model
        best_model_path = os.path.join(config_path, 'best_model.pth')
        if not os.path.exists(best_model_path):
            continue

        try:
            # Carica modello
            checkpoint = torch.load(best_model_path, map_location='cpu')
            config = checkpoint['config']['model_config']
            model = model_class(config)

            # Conta parametri
            total_params = sum(p.numel() for p in model.parameters())

            # Carica history per best accuracy e num epochs
            history_path = os.path.join(config_path, 'history.json')
            with open(history_path, 'r') as f:
                history = json.load(f)

            best_val_acc = max(history['val_acc'])
            num_epochs = len(history['train_loss'])

            # Stampa riga
            print(f"{config_name:<20} | {total_params:>15,} | {best_val_acc:>12.2f}% | {num_epochs:>7}")

            results.append({
                'config': config_name,
                'params': total_params,
                'best_acc': best_val_acc,
                'epochs': num_epochs
            })

        except Exception as e:
            print(f"{config_name:<20} | {'ERROR':>15} | {'-':>13} | {'-':>7}")

    print("="*90)

    # Trova best model
    if results:
        best = max(results, key=lambda x: x['best_acc'])
        print(f"\nBest Model: {best['config']} with {best['best_acc']:.2f}% accuracy")
        print(f"   Parameters: {best['params']:,}")
        print(f"   Epochs trained: {best['epochs']}")

In [ ]:
print_models_summary(
    model_class=create_cnn_model,
    checkpoint_dir='./checkpoints/cnn_experiments'
)


SUMMARY - All Trained Models
Config               |      Parameters |  Best Val Acc |  Epochs
------------------------------------------------------------------------------------------
cnn_baseline         |      17,679,989 |        35.38% |      15
cnn_deep             |      18,910,069 |        54.57% |      20
cnn_deep_residual    |       4,746,293 |        14.10% |      30
cnn_gap              |       1,311,861 |        13.06% |       8
cnn_large_kernels    |      17,845,365 |        32.25% |      15
cnn_optimized        |       3,820,661 |        13.29% |      17
cnn_shallow          |      34,383,093 |        38.38% |      12

Best Model: cnn_deep with 54.57% accuracy
   Parameters: 18,910,069
   Epochs trained: 20


Suppongo che i modello piu' profondi siamo andati peggio perche' non abbastanza dati

Grafico overfitting salvato: ./checkpoints/cnn_experiments/overfitting_analysis.png


OVERFITTING ANALYSIS
Config                    |  Train Acc |    Val Acc |        Gap | Status         
--------------------------------------------------------------------------------
cnn_optimized             |     12.72% |     12.72% |      0.00% | Good           
cnn_deep_residual         |     13.67% |     13.99% |     -0.31% | Good           
cnn_gap                   |     12.60% |     12.95% |     -0.34% | Good           
cnn_deep                  |     51.92% |     54.57% |     -2.64% | Good           
cnn_baseline              |     28.82% |     35.38% |     -6.55% | Good           
cnn_large_kernels         |     25.70% |     32.25% |     -6.55% | Good           
Note: Gap = Train Acc - Val Acc
  Gap > 15% = Severe overfitting
  Gap 5-15% = Moderate overfitting
  Gap < 5%  = Good generalization



In [ ]:
plot_overfitting_analysis(
    checkpoint_dir='./checkpoints/cnn_experiments',
    save_path='./checkpoints/cnn_experiments/overfitting_analysis.png'
)

# Plot diverse History

In [ ]:
def plot_all_histories(checkpoint_dir='./checkpoints/cnn_experiments', save_path=None):
    """
    Plotta tutte le history delle diverse configurazioni per confronto

    Args:
        checkpoint_dir: directory con le sottocartelle delle config
        save_path: (opzionale) path dove salvare il grafico
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Trova tutte le configurazioni
    config_names = [d for d in os.listdir(checkpoint_dir)
                   if os.path.isdir(os.path.join(checkpoint_dir, d))]

    # Colori per ogni config
    colors = plt.cm.tab10(np.linspace(0, 1, len(config_names)))

    for idx, config_name in enumerate(sorted(config_names)):
        history_path = os.path.join(checkpoint_dir, config_name, 'history.json')

        if not os.path.exists(history_path):
            print(f"Warning: history.json non trovato per {config_name}")
            continue

        # Carica history
        with open(history_path, 'r') as f:
            history = json.load(f)

        epochs = range(1, len(history['train_loss']) + 1)
        color = colors[idx]

        # Plot Train Loss
        axes[0, 0].plot(epochs, history['train_loss'],
                       color=color, marker='o', linewidth=2,
                       label=config_name, alpha=0.8)

        # Plot Val Loss
        axes[0, 1].plot(epochs, history['val_loss'],
                       color=color, marker='s', linewidth=2,
                       label=config_name, alpha=0.8)

        # Plot Train Accuracy
        axes[1, 0].plot(epochs, history['train_acc'],
                       color=color, marker='o', linewidth=2,
                       label=config_name, alpha=0.8)

        # Plot Val Accuracy
        axes[1, 1].plot(epochs, history['val_acc'],
                       color=color, marker='s', linewidth=2,
                       label=config_name, alpha=0.8)

    # Configurazione assi
    axes[0, 0].set_xlabel('Epoch', fontsize=12)
    axes[0, 0].set_ylabel('Loss', fontsize=12)
    axes[0, 0].set_title('Training Loss - All Configs', fontsize=14, fontweight='bold')
    axes[0, 0].legend(fontsize=9, loc='best')
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].set_xlabel('Epoch', fontsize=12)
    axes[0, 1].set_ylabel('Loss', fontsize=12)
    axes[0, 1].set_title('Validation Loss - All Configs', fontsize=14, fontweight='bold')
    axes[0, 1].legend(fontsize=9, loc='best')
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].set_xlabel('Epoch', fontsize=12)
    axes[1, 0].set_ylabel('Accuracy (%)', fontsize=12)
    axes[1, 0].set_title('Training Accuracy - All Configs', fontsize=14, fontweight='bold')
    axes[1, 0].legend(fontsize=9, loc='best')
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].set_xlabel('Epoch', fontsize=12)
    axes[1, 1].set_ylabel('Accuracy (%)', fontsize=12)
    axes[1, 1].set_title('Validation Accuracy - All Configs', fontsize=14, fontweight='bold')
    axes[1, 1].legend(fontsize=9, loc='best')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Grafico confronto salvato: {save_path}")

    plt.show()


def plot_final_comparison(checkpoint_dir='./checkpoints/cnn_experiments', save_path=None):
    """
    Crea un bar plot con le best validation accuracy di tutte le config

    Args:
        checkpoint_dir: directory con le sottocartelle delle config
        save_path: (opzionale) path dove salvare il grafico
    """
    config_names = []
    best_accs = []

    # Raccogli tutte le best accuracy
    for config_name in sorted(os.listdir(checkpoint_dir)):
        config_path = os.path.join(checkpoint_dir, config_name)
        if not os.path.isdir(config_path):
            continue

        history_path = os.path.join(config_path, 'history.json')
        if not os.path.exists(history_path):
            continue

        with open(history_path, 'r') as f:
            history = json.load(f)

        best_acc = max(history['val_acc'])
        config_names.append(config_name)
        best_accs.append(best_acc)

    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))

    bars = ax.bar(range(len(config_names)), best_accs,
                   color=plt.cm.viridis(np.linspace(0, 1, len(config_names))),
                   edgecolor='black', linewidth=1.5)

    # Aggiungi valori sopra le barre
    for i, (name, acc) in enumerate(zip(config_names, best_accs)):
        ax.text(i, acc + 0.5, f'{acc:.2f}%',
               ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.set_xlabel('Configuration', fontsize=13, fontweight='bold')
    ax.set_ylabel('Best Validation Accuracy (%)', fontsize=13, fontweight='bold')
    ax.set_title('Comparison of Best Validation Accuracy - All Configurations',
                fontsize=15, fontweight='bold')
    ax.set_xticks(range(len(config_names)))
    ax.set_xticklabels(config_names, rotation=45, ha='right')
    ax.grid(True, axis='y', alpha=0.3)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Grafico confronto finale salvato: {save_path}")

    plt.show()

Grafico confronto salvato: ./checkpoints/cnn_experiments/all_histories_comparison.png


Grafico confronto finale salvato: ./checkpoints/cnn_experiments/final_comparison.png


In [ ]:
plot_all_histories(
        checkpoint_dir='./checkpoints/cnn_experiments',
        save_path='./checkpoints/cnn_experiments/all_histories_comparison.png'
    )

# Plotta bar chart con best accuracies
plot_final_comparison(
    checkpoint_dir='./checkpoints/cnn_experiments',
    save_path='./checkpoints/cnn_experiments/final_comparison.png'
)

# Dataset piu' grande: CelebA, con kaggle

In [ ]:
def setup_kaggle_and_download_celeba(kaggle_username=None, kaggle_key=None,
                                     data_dir='./content/data/celeba'):
    """
    Setup Kaggle e scarica CelebA dataset

    PREREQUISITI:
    1. Vai su https://www.kaggle.com/settings
    2. Scroll down a "API" section
    3. Click "Create New API Token"
    4. Copia USERNAME e KEY dal token mostrato

    Args:
        kaggle_username: tuo username Kaggle
        kaggle_key: tuo API key Kaggle
        data_dir: directory dove estrarre il dataset

    Returns:
        path del dataset estratto
    """
    import subprocess
    import os
    import json

    print("="*70)
    print("SETUP KAGGLE API")
    print("="*70)

    # Installa kaggle
    print("\n1. Installazione Kaggle API...")
    subprocess.run(['pip', 'install', '-q', 'kaggle'], check=True)
    print("   ✓ Kaggle API installato")

    # Setup credenziali
    print("\n2. Setup credenziali...")
    kaggle_dir = os.path.expanduser('~/.kaggle')
    os.makedirs(kaggle_dir, exist_ok=True)

    # Opzione 1: usa username e key forniti
    if kaggle_username and kaggle_key:
        print("   ✓ Credenziali fornite direttamente")
        kaggle_json = {
            "username": kaggle_username,
            "key": kaggle_key
        }

        with open(f'{kaggle_dir}/kaggle.json', 'w') as f:
            json.dump(kaggle_json, f)

        subprocess.run(['chmod', '600', f'{kaggle_dir}/kaggle.json'], check=True)
        print("   ✓ Credenziali configurate")

    # Opzione 2: cerca kaggle.json già uploadato
    elif os.path.exists('/content/kaggle.json'):
        print("   ✓ kaggle.json trovato!")
        subprocess.run(['cp', '/content/kaggle.json', f'{kaggle_dir}/kaggle.json'], check=True)
        subprocess.run(['chmod', '600', f'{kaggle_dir}/kaggle.json'], check=True)
        print("   ✓ Credenziali configurate")

    else:
        print("\n    ERRORE: Credenziali non fornite!")
        print("\n   OPZIONE 1 - Passa username e key:")
        print("   setup_kaggle_and_download_celeba(")
        print("       kaggle_username='tuo_username',")
        print("       kaggle_key='tua_api_key'")
        print("   )")
        print("\n   OPZIONE 2 - Upload kaggle.json:")
        print("   1. Vai su https://www.kaggle.com/settings")
        print("   2. Scroll to 'API' section")
        print("   3. Click 'Create New API Token'")
        print("   4. Download kaggle.json")
        print("   5. Upload kaggle.json su Colab (sidebar Files)")
        print("   6. Riprova: setup_kaggle_and_download_celeba()")
        return None

    # Download dataset
    print("\n3. Download CelebA dataset...")
    print("   (Questo richiederà 3-5 minuti, ~1.5GB)")

    os.makedirs(data_dir, exist_ok=True)

    try:
        # Download
        subprocess.run([
            'kaggle', 'datasets', 'download', '-d',
            'jessicali9530/celeba-dataset',
            '-p', data_dir
        ], check=True)
        print("   ✓ Download completato!")

        # Unzip
        print("\n4. Estrazione archivio...")
        zip_file = os.path.join(data_dir, 'celeba-dataset.zip')
        subprocess.run(['unzip', '-q', zip_file, '-d', data_dir], check=True)

        # Rimuovi zip
        os.remove(zip_file)
        print("   ✓ Dataset estratto!")

        print("\n" + "="*70)
        print("✓ CELEBA PRONTO!")
        print(f"  Path: {data_dir}")
        print("="*70 + "\n")

        return data_dir

    except subprocess.CalledProcessError as e:
        print(f"\n    Errore durante download: {e}")
        print("   Verifica le credenziali Kaggle")
        return None

def load_celeba_from_kaggle(data_dir='/content/data/celeba', img_size=128,
                            batch_size=64, augment=True, test_split=0.2,
                            max_samples=10000):
    """
    Carica CelebA da dataset Kaggle già scaricato

    Args:
        data_dir: directory del dataset CelebA
        img_size: dimensione immagini
        batch_size: batch size
        augment: data augmentation
        test_split: split test
        min_images_per_person: minimo immagini per identità
        max_images_per_person: massimo immagini per identità (per bilanciare)

    Returns:
        train_loader, test_loader, num_classes
    """
    import pandas as pd
    from collections import Counter

    print("Caricamento CelebA da Kaggle...")

    # Path del dataset
    img_dir = os.path.join(data_dir, 'img_align_celeba', 'img_align_celeba')
    attr_file = os.path.join(data_dir, 'list_attr_celeba.csv')

    # Controlla se esiste
    if not os.path.exists(img_dir):
        print(f"❌ Immagini non trovate in {img_dir}")
        return None, None, None

    if not os.path.exists(attr_file):
        print(f"❌ Attributi non trovati in {attr_file}")
        return None, None, None

    print("  ✓ Dataset trovato!")

    # Leggi attributi
    print("  Lettura attributi...")
    df_attr = pd.read_csv(attr_file)

    # Lista tutte le immagini
    all_images = sorted(os.listdir(img_dir))

    # Limita se richiesto
    if max_samples and len(all_images) > max_samples:
        print(f"  Limitando a {max_samples} samples (su {len(all_images)} totali)")
        all_images = all_images[:max_samples]

    print(f"  Samples usati: {len(all_images)}")

    # Crea labels artificiali basate su combinazione attributi comuni
    # Usiamo: Male, Young, Smiling, Eyeglasses → 2^4 = 16 classi
    print("  Creazione classi da attributi...")

    images_paths = []
    labels = []

    for img_name in all_images:
        if img_name not in df_attr['image_id'].values:
            continue

        img_path = os.path.join(img_dir, img_name)

        # Estrai attributi per questa immagine
        row = df_attr[df_attr['image_id'] == img_name].iloc[0]

        # Crea label da 4 attributi principali (binari)
        male = 1 if row.get('Male', 0) == 1 else 0
        young = 1 if row.get('Young', 0) == 1 else 0
        smiling = 1 if row.get('Smiling', 0) == 1 else 0
        eyeglasses = 1 if row.get('Eyeglasses', 0) == 1 else 0

        # Combina in label unico (0-15)
        label = male * 8 + young * 4 + smiling * 2 + eyeglasses

        images_paths.append(img_path)
        labels.append(label)

    num_classes = len(set(labels))
    print(f"  Classi create: {num_classes}")

    from collections import Counter
    label_dist = Counter(labels)
    print(f"  Distribuzione (top 5): {label_dist.most_common(5)}")

    # Split train/test stratificato
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        images_paths, labels,
        test_size=test_split,
        stratify=labels,
        random_state=42
    )

    # Crea transforms
    train_transform, test_transform = get_data_transforms(img_size, augment)

    # Custom dataset per CelebA da paths
    class CelebAPathDataset(Dataset):
        def __init__(self, image_paths, labels, transform):
            self.image_paths = image_paths
            self.labels = labels
            self.transform = transform

        def __len__(self):
            return len(self.image_paths)

        def __getitem__(self, idx):
            img = Image.open(self.image_paths[idx]).convert('RGB')
            label = self.labels[idx]

            if self.transform:
                img = self.transform(img)

            return img, label

    train_dataset = CelebAPathDataset(train_paths, train_labels, train_transform)
    test_dataset = CelebAPathDataset(test_paths, test_labels, test_transform)

    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    print(f"\n✓ DataLoaders CelebA pronti:")
    print(f"  Train samples: {len(train_dataset)}")
    print(f"  Test samples: {len(test_dataset)}")
    print(f"  Num classes: {num_classes}")
    print(f"  Batch size: {batch_size}")
    print(f"\n  NOTA: Classi create da attributi (Male/Young/Smiling/Eyeglasses)")
    print(f"        Non sono identità reali ma categorie artificiali\n")

    return train_loader, test_loader, num_classes

setup Kaggle

In [ ]:
setup_kaggle_and_download_celeba(
    kaggle_username='mrmoro',
    kaggle_key='KGAT_52381a474ad3b79ffab01ccf5632c3bc'
)

SETUP KAGGLE API

1. Installazione Kaggle API...
   ✓ Kaggle API installato

2. Setup credenziali...
   ✓ Credenziali fornite direttamente
   ✓ Credenziali configurate

3. Download CelebA dataset...
   (Questo richiederà 3-5 minuti, ~1.5GB)
   ✓ Download completato!

4. Estrazione archivio...
   ✓ Dataset estratto!

✓ CELEBA PRONTO!
  Path: ./content/data/celeba



'./content/data/celeba'

Scarica il dataset

In [ ]:
train_loader, test_loader, num_classes = load_celeba_from_kaggle(
    data_dir='/content/content/data/celeba',
    img_size=128,
    batch_size=128,
    max_samples=202599  # Limita a 10k per velocità
)

Caricamento CelebA da Kaggle...
  ✓ Dataset trovato!
  Lettura attributi...
  Samples usati: 202599
  Creazione classi da attributi...
  Classi create: 16
  Distribuzione (top 5): [(6, 52962), (4, 48603), (12, 30405), (14, 19260), (8, 13770)]

✓ DataLoaders CelebA pronti:
  Train samples: 162079
  Test samples: 40520
  Num classes: 16
  Batch size: 128

  NOTA: Classi create da attributi (Male/Young/Smiling/Eyeglasses)
        Non sono identità reali ma categorie artificiali



# Modello deep con skip conenction con dataset grande

## Ricreo config cnn con quelle piu' interessanti

In [ ]:
def get_cnn_configs():
    """
    Configurazioni CNN dalla più semplice alla più ottimizzata

    Returns:
        dict con tutte le configurazioni
    """
    configs = {
        # 1. Baseline CNN - configurazione base
        'cnn_baseline': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64, 128],  # 3 conv blocks
                'kernel_sizes': [3, 3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.5,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 15,
                'optimizer': 'adam',
                'weight_decay': 0.0,
                'label_smoothing': 0.0,
                'early_stopping_patience': 6
               }
        },

        # 2. Shallow CNN - meno layer convoluzionali
        'cnn_shallow': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64],  # Solo 2 conv blocks
                'kernel_sizes': [3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 12,
                'optimizer': 'adam',
                'weight_decay': 0.0,
                'label_smoothing': 0.0,
                'early_stopping_patience': 5
            }
        },

        # 3. Deep CNN - più layer convoluzionali
        'cnn_deep': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [32, 64, 128, 256],  # 4 conv blocks
                'kernel_sizes': [3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': False
            },
            'training_config': {
                'learning_rate': 5e-4,  # LR più basso per deep
                'batch_size': 32,  # Batch più piccolo
                'num_epochs': 20,
                'optimizer': 'adam',
                'weight_decay': 1e-5,
                'label_smoothing': 0.05,
                'early_stopping_patience': 7
            }
        },

        # 5. Global Average Pooling - riduce parametri
        'cnn_gap': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 128, 256],
                'kernel_sizes': [3, 3, 3],
                'fc_hidden_size': 512,
                'embedding_size': 128,
                'dropout_rate': 0.4,
                'use_batchnorm': True,
                'use_global_avg_pool': True  # Usa GAP
            },
            'training_config': {
                'learning_rate': 1e-3,
                'batch_size': 64,
                'num_epochs': 15,
                'optimizer': 'adam',
                'weight_decay': 1e-5,
                'label_smoothing': 0.0,
                'early_stopping_patience': 6
            }
        },

        # 6. Optimized CNN - best practices
        'cnn_optimized': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 128, 256, 512],  # Progressione standard
                'kernel_sizes': [3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.3,
                'use_batchnorm': True,
                'use_global_avg_pool': True,  # GAP per ridurre parametri
                'use_residual': False
            },
            'training_config': {
                'learning_rate': 5e-4,
                'batch_size': 64,
                'num_epochs': 25,
                'optimizer': 'adam',
                'weight_decay': 1e-4,  # L2 reg
                'label_smoothing': 0.1,  # Label smoothing
                'early_stopping_patience': 8
            }
        },

        # 7. Deep Residual CNN - rete profonda con skip connections
        'cnn_deep_residual': {
            'model_config': {
                'input_channels': 3,
                'num_filters': [64, 64, 128, 128, 256, 256],  # 6 residual blocks
                'kernel_sizes': [3, 3, 3, 3, 3, 3],
                'fc_hidden_size': 1024,
                'embedding_size': 256,
                'dropout_rate': 0.3,
                'use_batchnorm': True,
                'use_global_avg_pool': True,
                'use_residual': True  # Skip connections attive
            },
            'training_config': {
                'learning_rate': 3e-4,  # LR più basso per deep residual
                'batch_size': 32,
                'num_epochs': 30,
                'optimizer': 'adam',
                'weight_decay': 1e-4,
                'label_smoothing': 0.1,
                'early_stopping_patience': 10
            }
        }
    }

    return configs

## CNN su CelebA

> Importante: le classi sono cambiate, quindi i valori potrebbero piú alti non necessariamente significano una rete davvero migliore <br>
> Cambiato classi perché il dataset forniva tali labels



In [ ]:
# Train tutte le configurazioni
results = train_multiple_configs(
    model_class=create_cnn_model,
    train_loader=train_loader,
    test_loader=test_loader,
    num_classes=num_classes,
    checkpoint_dir='./checkpoints/cnn_celeba_experiments',
    device=None
)


Starting training: cnn_baseline


TRAINING: cnn_baseline
Model config: {'input_channels': 3, 'num_filters': [32, 64, 128], 'kernel_sizes': [3, 3, 3], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.5, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 16}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 15, 'optimizer': 'adam', 'weight_decay': 0.0, 'label_smoothing': 0.0, 'early_stopping_patience': 6}


Epoch 1/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:53<00:00,  5.97it/s, loss=0.9747, acc=71.96%]



Epoch 1 Summary:
  Train Loss: 1.2196 | Train Acc: 57.65%
  Val Loss: 0.7697 | Val Acc: 71.96%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 71.96% ***

Epoch 2/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:54<00:00,  5.87it/s, loss=0.8592, acc=74.41%]



Epoch 2 Summary:
  Train Loss: 0.8760 | Train Acc: 68.48%
  Val Loss: 0.6930 | Val Acc: 74.41%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 74.41% ***

Epoch 3/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.10it/s, loss=0.8052, acc=76.34%]



Epoch 3 Summary:
  Train Loss: 0.8095 | Train Acc: 70.46%
  Val Loss: 0.6404 | Val Acc: 76.34%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 76.34% ***

Epoch 4/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.21it/s, loss=0.7383, acc=77.22%]



Epoch 4 Summary:
  Train Loss: 0.7675 | Train Acc: 71.92%
  Val Loss: 0.6094 | Val Acc: 77.22%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 77.22% ***

Epoch 5/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.13it/s, loss=0.8037, acc=77.78%]



Epoch 5 Summary:
  Train Loss: 0.7405 | Train Acc: 72.84%
  Val Loss: 0.5983 | Val Acc: 77.78%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 77.78% ***

Epoch 6/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.02it/s, loss=0.7449, acc=77.99%]



Epoch 6 Summary:
  Train Loss: 0.7235 | Train Acc: 73.38%
  Val Loss: 0.5838 | Val Acc: 77.99%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 77.99% ***

Epoch 7/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:57<00:00,  5.49it/s, loss=0.7159, acc=78.53%]



Epoch 7 Summary:
  Train Loss: 0.7077 | Train Acc: 73.92%
  Val Loss: 0.5749 | Val Acc: 78.53%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 78.53% ***

Epoch 8/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.00it/s, loss=0.7470, acc=78.69%]



Epoch 8 Summary:
  Train Loss: 0.6966 | Train Acc: 74.37%
  Val Loss: 0.5705 | Val Acc: 78.69%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 78.69% ***

Epoch 9/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.07it/s, loss=0.7148, acc=78.94%]



Epoch 9 Summary:
  Train Loss: 0.6848 | Train Acc: 74.68%
  Val Loss: 0.5623 | Val Acc: 78.94%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 78.94% ***

Epoch 10/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:55<00:00,  5.74it/s, loss=0.7352, acc=79.12%]



Epoch 10 Summary:
  Train Loss: 0.6755 | Train Acc: 74.96%
  Val Loss: 0.5574 | Val Acc: 79.12%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 79.12% ***

Epoch 11/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.01it/s, loss=0.7439, acc=78.80%]



Epoch 11 Summary:
  Train Loss: 0.6681 | Train Acc: 75.24%
  Val Loss: 0.5594 | Val Acc: 78.80%
  EarlyStopping counter: 1/6

Epoch 12/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.01it/s, loss=0.7457, acc=79.23%]



Epoch 12 Summary:
  Train Loss: 0.6595 | Train Acc: 75.53%
  Val Loss: 0.5560 | Val Acc: 79.23%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 79.23% ***

Epoch 13/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:53<00:00,  5.96it/s, loss=0.7161, acc=79.55%]



Epoch 13 Summary:
  Train Loss: 0.6546 | Train Acc: 75.69%
  Val Loss: 0.5412 | Val Acc: 79.55%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 79.55% ***

Epoch 14/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.14it/s, loss=0.7160, acc=79.66%]



Epoch 14 Summary:
  Train Loss: 0.6475 | Train Acc: 76.01%
  Val Loss: 0.5378 | Val Acc: 79.66%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 79.66% ***

Epoch 15/15
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.11it/s, loss=0.7330, acc=79.82%]



Epoch 15 Summary:
  Train Loss: 0.6427 | Train Acc: 76.00%
  Val Loss: 0.5371 | Val Acc: 79.82%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/best_model.pth
  *** Nuovo best model! Accuracy: 79.82% ***
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_baseline/last_checkpoint.pth

TRAINING COMPLETATO: cnn_baseline
Best Validation Accuracy: 79.82%


Starting training: cnn_shallow


TRAINING: cnn_shallow
Model config: {'input_channels': 3, 'num_filters': [32, 64], 'kernel_sizes': [3, 3], 'fc_hidden_size': 512, 'embedding_size': 128, 'dropout_rate': 0.4, 'use_batchnorm': True, 'use_global_avg_pool': False, 'num_classes': 16}
Training config: {'learning_rate': 0.001, 'batch_size': 64, 'num_epochs': 12, 'optimizer': 'adam', 'weight_decay': 0.0, 'label_smoothing': 0.0, 'early_stopping_patience': 5}


Epoch 1/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.11it/s, loss=0.8859, acc=74.92%]



Epoch 1 Summary:
  Train Loss: 1.0522 | Train Acc: 63.31%
  Val Loss: 0.6877 | Val Acc: 74.92%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 74.92% ***

Epoch 2/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:51<00:00,  6.13it/s, loss=0.7769, acc=76.17%]



Epoch 2 Summary:
  Train Loss: 0.8011 | Train Acc: 70.81%
  Val Loss: 0.6420 | Val Acc: 76.17%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 76.17% ***

Epoch 3/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.05it/s, loss=0.8073, acc=77.45%]



Epoch 3 Summary:
  Train Loss: 0.7428 | Train Acc: 72.84%
  Val Loss: 0.6083 | Val Acc: 77.45%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 77.45% ***

Epoch 4/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  5.99it/s, loss=0.7282, acc=78.00%]



Epoch 4 Summary:
  Train Loss: 0.7044 | Train Acc: 74.09%
  Val Loss: 0.5899 | Val Acc: 78.00%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 78.00% ***

Epoch 5/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:53<00:00,  5.98it/s, loss=0.7149, acc=78.93%]



Epoch 5 Summary:
  Train Loss: 0.6799 | Train Acc: 74.85%
  Val Loss: 0.5663 | Val Acc: 78.93%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 78.93% ***

Epoch 6/12
----------------------------------------------------------------------


Validation: 100%|██████████| 317/317 [00:52<00:00,  6.09it/s, loss=0.7087, acc=79.11%]



Epoch 6 Summary:
  Train Loss: 0.6627 | Train Acc: 75.43%
  Val Loss: 0.5588 | Val Acc: 79.11%
Checkpoint salvato: ./checkpoints/cnn_celeba_experiments/cnn_shallow/best_model.pth
  *** Nuovo best model! Accuracy: 79.11% ***

Epoch 7/12
----------------------------------------------------------------------


Training:  10%|█         | 127/1267 [00:37<05:02,  3.77it/s, loss=0.6197, acc=75.56%]

# Download Files and Models

In [ ]:
def create_results_zip(checkpoint_dir='./checkpoints',
                       output_zip='training_results.zip',
                       exclude_dataset=True,
                       only_best_config=None):
    """
    Crea un file zip con tutti i risultati (modelli, config, history, grafici)
    escludendo il dataset

    Args:
        checkpoint_dir: directory da zippare
        output_zip: nome del file zip di output
        exclude_dataset: se True, esclude la cartella data/
        only_best_config: se specificato (es. 'optimized'), salva solo i modelli
                         di quella config, ma history/grafici di tutte

    Returns:
        path del file zip creato
    """
    print(f"Creazione zip: {output_zip}")
    print("Compressione in corso...")

    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(checkpoint_dir):
            # Escludi dataset se richiesto
            if exclude_dataset and 'data' in root:
                continue

            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(checkpoint_dir))

                # Se only_best_config è specificato
                if only_best_config:
                    # Salta i .pth di config diverse da quella best
                    if file.endswith('.pth'):
                        if only_best_config not in root:
                            print(f"  Saltato (non best config): {arcname}")
                            continue

                zipf.write(file_path, arcname)
                print(f"  Aggiunto: {arcname}")

    file_size = os.path.getsize(output_zip) / (1024 * 1024)  # MB
    print(f"\nZip creato: {output_zip}")
    print(f"Dimensione: {file_size:.2f} MB")
    print("\nPer scaricare su Colab:")
    print(f"  from google.colab import files")
    print(f"  files.download('{output_zip}')")

    return output_zip


def download_results_colab(checkpoint_dir='./checkpoints',
                          zip_name='training_results.zip',
                          only_best_config=None):
    """
    Crea zip e avvia download automatico su Google Colab

    Args:
        checkpoint_dir: directory da zippare
        zip_name: nome del file zip
        only_best_config: se specificato (es. 'optimized'), salva solo i modelli
                         di quella config, ma history/grafici di tutte
    """
    # Crea zip
    zip_path = create_results_zip(checkpoint_dir, zip_name,
                                  exclude_dataset=True,
                                  only_best_config=only_best_config)

    # Download automatico su Colab
    try:
        from google.colab import files
        print("\nAvvio download...")
        files.download(zip_path)
        print("Download completato!")
    except ImportError:
        print("\nNOTA: Non sei su Google Colab.")
        print(f"Il file zip è disponibile in: {zip_path}")

In [ ]:
download_results_colab(
    checkpoint_dir='./checkpoints/cnn_celeba_experiments',
    zip_name='dnn_results_light.zip',
)

NameError: name 'download_results_colab' is not defined

In [ ]:
plot_all_histories(
        checkpoint_dir='./checkpoints/cnn_celeba_experiments',
        save_path='./checkpoints/cnn_celeba_experiments/all_histories_comparison.png'
    )

# Plotta bar chart con best accuracies
plot_final_comparison(
    checkpoint_dir='./checkpoints/cnn_celebaexperiments',
    save_path='./checkpoints/cnn_celebaexperiments/final_comparison.png'
)

In [ ]:
print_models_summary(
    model_class=create_cnn_model,
    checkpoint_dir='./checkpoints/cnn_celeba_experiments'
)

In [ ]:
plot_overfitting_analysis(
    checkpoint_dir='./checkpoints/cnn_celeba_experiments',
    save_path='./checkpoints/cnn_celeba_experiments/overfitting_analysis.png'
)